<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/203_NumPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NumPy
=====

NumPy の概要
------------

[NumPy](https://numpy.org/ja/) は、Python で数値計算を行うためのライブラリである。 NumPy の内部は C と FORTRAN によって実装されているため非常に高速に動作する（Python での GIL による制限を受けずに並列処理が可能である）。ライセンスは BSD 3-clause License。

2024 年 6 月 16 日に NumPy 2.0 がリリースされ、NumPy 1.xx で書かれた既存のコードに影響する API 変更が行われた。サポートされる Python のバージョンは以下のとおり。

| NumPy | サポートされる Python |
|:--:|:--:|
| NumPy 2.1.x | Python 3.10-3.13 |
| NumPy 2.0.x | Python 3.9-3.12 |
| NumPy 1.26.x | Python 3.9-3.12 |
| NumPy 1.24.x | Python 3.8-3.11 |

このノートでは、 NumPy 1.26.x を扱い、 NumPy 2.0 での変更点を追記する形を取る。

NumPy 1.26.x のインストール方法は次のとおり。

``` shell
pip install numpy~=1.26.0
```

Ruff で NumPy 2.0 との互換性をチェックすることができる。このチェック機能を有効にするには、 `pyproject.toml` に `NPY201` ルールを追加する必要がある。

``` ini
[tool.ruff.lint]
extend-select = ["NPY201"]
```

<font color="red">このノートのコードは、すべて NumPy が以下のようにインポートされていることを前提とする（コードセルを実行する場合は最初に次のコードでインポートを済ませる必要がある）。</font>

In [4]:
import numpy as np


モジュール定数
--------------

NumPy は、以下のモジュール定数を提供する。

| 定数 | 意味 |
|:---|:---|
| `numpy.__version__` | NumPy のバージョン |
| `numpy.nan` | 非数の IEEE 754 浮動小数点表現 |
| `numpy.inf` | 無限大の IEEE 754 浮動小数点表現 |
| `numpy.e` | ネイピア数（自然対数の底） |
| `numpy.pi` | $\pi$（円周率） |
| `numpy.euler_gamma` | [オイラーの $\gamma$ 定数](https://ja.wikipedia.org/wiki/オイラーの定数) |
| `numpy.newaxis` | `None` の便利な別名（エイリアス） |

なお、 NumPy のバージョンは、 `numpy.version` サブモジュールの定数 `version` でも確認できる。

In [ ]:
np.__version__, np.version.version

('1.26.4', '1.26.4')

In [ ]:
np.nan, np.inf

(nan, inf)

In [ ]:
np.e, np.pi, np.euler_gamma

(2.718281828459045, 3.141592653589793, 0.5772156649015329)

In [ ]:
np.newaxis is None

True

多次元配列
----------

### 多次元配列の特徴 ###

`numpy.ndarray` クラスは、**多次元配列**（N-dimensional array）を扱う。多次元配列は、数学の概念で言えば、 1 次元の場合は**ベクトル**（vector）に、 2 次元の場合は**行列**（matrix）に、 3 次元以上の場合は**テンソル**（tensor）に該当するものである。 NumPy を使わない場合、 Python ではこうした多次元配列を表現するにはネストされたリスト（多重リスト）が利用される。 `np.ndarray` は Python のネストされたリストとは異なる独自のデータ構造であり、以下のような特徴を持つ。

  * **型の同一性**: 同じデータ型の要素しか格納することができない。データ型は原則として固定長である必要がある。
  * **固定サイズ**: 各次元ごと（2 次元なら列ごとや行ごと）の配列の大きさは必ず一定である。

多次元配列の各次元の配列の固定された大きさの組み合わせを**形状**（shape）と呼ぶ。 NumPy では多次元配列の形状を、次元の小さい順で並べたタプルで表す。

多次元配列の形状を考える上で、**軸**（axis）という概念が重要である。軸には 0 から始まる番号が付く。 Python のネストされたリストで要素をインデックス参照する場合に `[i][j][k]...` のように `[]` を並べ外側のリストから内側に向かってインデックス参照を重ねる形になるのであるが、この順番が多次元配列の軸の番号に相当する。 2 次元配列の場合、 0 軸は行方向であり、 1 軸は列方向となる。

``` text
          1 軸
           →
       [[1, 2, 3],
0 軸 ↓ [4, 5, 6],
        [7, 8, 9]]
```

`n` 軸は `n+1` 次元の配列に対応している。したがって、 2 次元配列の場合、形状が `(i, j)` であるとは、行の数が全ての列で `i` に固定され、列の数が全ての行で `j` に固定されることを意味する。

### 配列の要素の型 ###

NumPy のデータ型は dtype と呼ばれる。

dtype は型オブジェクトであり、次の2種類のコンストラクタ呼び出しにより作成される。

``` python
numpy.dtype(型名の文字列)
numpy.dtype(型コードの文字列)
```

一方、NumPy は指定可能な型名による dtype の定数を用意している。これらは C で使用可能な型に基づいている。

型名、型コード、dtype 定数の関係は以下の通り。

| dtype 定数 | 型名 | 型コード | 意味 |
|:---|:---|:---|:---|
| `numpy.int8` | `int8` | `'i1'` | 8 ビット符号あり整数型 （`-128` ～ `127`） |
| `numpy.int16` | `int16` | `'i2'` | 16 ビット符号あり整数型 （`-32_768` ～ `32_767`） |
| `numpy.int32` | `int32` | `'i4'` | 32 ビット符号あり整数型 （`-2_147_483_648` ～ `2_147_483_647`） |
| `numpy.int64` | `int64` | `'i8'` | 64 ビット符号あり整数型 （`-9_223_372_036_854_775_808` ～ `9_223_372_036_854_775_807`） |
| `numpy.uint8` | `uint8` | `'u1'` | 8 ビット符号なし整数型 （`0` ～ `255`） |
| `numpy.uint16` | `uint16` | `'u2'` | 16 ビット符号なし整数型 （`0` ～ `65_535`） |
| `numpy.uint32` | `uint32` | `'u4'` | 32 ビット符号なし整数型 （`0` ～ `4_294_967_295`） |
| `numpy.uint64` | `uint64` | `'u8'` | 64 ビット符号なし整数型 （`0` ～ `18_446_744_073_709_551_615`） |
| `numpy.float16`| `float16`| `'f2'` | 半精度浮動小数点数型 （符号部 `1` ビット、指数部 `5` ビット、仮数部 `10` ビット） |
| `numpy.float32` | `float32` | `'f4'` | 単精度浮動小数点型 （符号部 `1` ビット、指数部 `8` ビット、仮数部 `23` ビット） |
| `numpy.float64` | `float64` | `'f8'` | 倍精度浮動小数点型 （符号部 `1` ビット、指数部 `11` ビット、仮数部 `52` ビット） |
| `numpy.float128` | `float128` | `'f16'` | 四倍精度浮動小数点型 （符号部 `1` ビット、指数部 `15`ビット、仮数部 `112` ビット） |
| `numpy.complex64` | `complex64` | `'c8'` | 複素数型 （実部・虚部がそれぞれ `numpy.float32`） |
| `numpy.complex128` | `complex128` | `'c16'` | 複素数型 （実部・虚部がそれぞれ `numpy.float64`） |
| `numpy.complex256` | `complex256` | `'c32'` | 複素数型 （実部・虚部がそれぞれ `numpy.float128`） |

In [ ]:
assert np.int64 == np.dtype('int64') == np.dtype('i8')

Numpy の関数やメソッドの引数で dtype を指定するとき、「型オブジェクトの定数」「型名の文字列」「型コードの文字列」の 3 通りで指定することができる。たとえば、 64 ビット符号あり整数型は以下の方法で指定可能である。

  * 型オブジェクトの定数: `numpy.int64`
  * 型名の文字列: `'int64'`
  * 型コードの文字列: `'i8'`

dtype の指定に「型名の文字列」「型コードの文字列」を使用すると、 Numpy の関数の内部で自動的に型オブジェクトに変換される。

整数型と浮動小数点数型についての情報は、それぞれ `numpy.iinfo` クラスと `numpy.finfo` クラスで確認できる。コンストラクタ引数として dtype を指定する。

`numpy.iinfo` は、整数型クラスについて最小値 `min`、最大値 `max`、ビット数 `bits`、型名 `dtype` の属性を持つクラスである。

`numpy.finfo` は、浮動小数点数型クラスについて最小値 `min`、最大値 `max`、精度 `resolution`、ビット数 `bits`、型名 `dtype` などの属性を持つクラスである。

In [ ]:
print(f"{np.iinfo('i8')!r}")  # print(f"{np.iinfo(np.int64)!r}") のように直接型オブジェクトを指定するのと同様
print(f"{np.finfo('f8')!r}")  # print(f"{np.iinfo(np.float64)!r}") のように直接型オブジェクトを指定するのと同様

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)
finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)


また、 Python の型と互換性のある追加の型がいくつかある。

| dtype 定数 | 型名 | 型コード | Python 型 | 備考 |
|:---|:---|:---|:---|:---|
| `numpy.int_` | `int` | `'l'` | `int` | 64 ビット版 Linux では `numpy.int64` と同じ。Windows および 32 ビット版 Linux では `numpy.int32` と同じ |
| `numpy.bool_` | `bool` | `'?'` | `bool` | ブール型。`True` か `False` で表される 8 ビットデータ |
| `numpy.float_` | `float` | `'d'` | `float` | `numpy.double` の別名で、`numpy.float64` と同じ |
| `numpy.complex_` | `complex` | `'D'` | `complex` | `numpy.cdouble` の別名で、`numpy.complex128` と同じ |
| `numpy.bytes_` | `bytes` | `'S'` | `bytes` | バイト列型 |
| `numpy.str_` | `unicode` | `'U'` | `str` | Unicode 文字列型。UCS-4 のコードポイントを表す 32 ビット単位のデータ |
| `numpy.datetime64` | `datetime64` | `'M'` | `datetime.datetime` | 日時型。64 ビット整数で Unix エポック秒を表す |
| `numpy.timedelta64` | `timedelta64` | `'m'` | `datetime.timedelta` | 時間差型。64 ビット整数で `timedelta` を表す |
| `numpy.object_` | `object` | `'O'` | Python オブジェクト型 |

これらの追加の型は、 dtype の指定で Python の型が使用された場合に、その Python の型を自動的に変換するために用意されている。

たとえば、 dtype の指定で Python の `int` を指定した場合、 `numpy.int_` 型の指定に変換される。ただし、 `numpy.int_` 型は、 Python の `int` 型とは少し異なる。 Python の `int` 型は任意長となる多倍長整数型であるが、 `np.ndarray` ではデータ型は固定長である必要があるため、 `numpy.int64` や `numpy.int32` といった固定長の整数型として扱われる。

In [ ]:
assert np.int_ == np.int64  # 64 ビット版 Linux では numpy.int64 と同じ

`numpy.bool_` 型は、 `numpy.int_` のサブクラスではなく、それとは別の型であること（Python で `bool` が `int` のサブクラスとされるのとは異なる扱い）に注意する。

In [ ]:
assert not issubclass(np.bool_, np.int_) and issubclass(bool, int)

dtype の指定で Python の `bytes` や `str` を指定した場合、 それぞれ `numpy.bytes_` 型や `numpy.str_` 型の指定に変換されるのであるが、 `numpy.bytes_` 型と `numpy.str_` 型も、それぞれ Python の `bytes` 型と `str` 型とは異なり固定長になる。また、エンディアンの指定も必要となる。最大長とエンディアンを「型コードの文字列」で指定するには、次の書式を使用する。

`numpy.bytes_` 型:

``` text
エンディアンS最大長
```

`numpy.str_` 型:

``` text
エンディアンU最大長
```

最大長は、 `bytes` 型ではバイト数であり、 `str` 型では文字数である。また、接頭辞でバイトオーダーを指定する。先頭の `'<'` でリトルエンディアンを、 `'>'` でビッグエンディアンを表す。たとえば、リトルエンディアンで最長 3 文字の `numpy.str_` 型は `'<U3'` となる。 `numpy.bytes_` にはエンディアンを適用できないので、適用不可を表す `'|'` のみを指定できる（例 `'|S3'`）。

【NumPy 2.0 での変更点】  
データ型が以下のように変更されている。

  * 64 ビット版 Windows でも、 `numpy.int64` が `numpy.int_` の別名とされた。
  * `numpy.double` が直接 Python の型 `float` と対応するようになり、 `numpy.float_` は削除された。
  * `numpy.cdouble` が直接 Python の型 `complex` と対応するようになり、 `numpy.complex_` は削除された。
  * UTF-8 の 可変長文字列型 `numpy.dtypes.StringDType` が追加された。

### 配列のメモリレイアウト ###

`numpy.ndarray` のインスタンスは、メモリ上では C や Fortran の配列と同様にメモリの連続領域上に 1 次元で保存される。その先頭には、データ型や形状といった、1 次元で並べられた要素のデータの読み取り方が登録される。ここに登録される情報を**メタデータ**と呼ぶ。

メタデータに続いて実際のデータ（要素のあつまり）が並んでいるのであるが、このデータの格納の仕方（**メモリレイアウト**）は 2 通りある。

  * **ローメジャーオーダー**（row-major order）： C で使われている並べ方で、低い次元（つまり軸の番号が小さい順）から並んでいるように格納していく。
  * **カラムメジャーオーダー**（column-major order）:  FORTRAN で使われている並べ方で、高い次元（つまり軸の番号が大きい順）から並んでいるように格納していく。

たとえば、2 次元配列では、ローメジャーオーダーなら要素は行が連続的にメモリ上に並んでいる形で、次に列が並んでいる形になる。また、カラムメジャーオーダーなら要素は列が連続的にメモリ上に並んでいる形で、次に行が並んでいる形になる。 3×3 の 2 次元配列では次のようになる（[Wikipedia](https://en.wikipedia.org/wiki/Row-_and_column-major_order)から引用）。

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Row_and_column_major_order.svg/180px-Row_and_column_major_order.svg.png)

``` text
Row-major order:
┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│ a_11 │ a_12 │ a_13 │ a_21 │ a_22 │ a_23 │ a_31 │ a_32 │ a_33 │
└───┴───┴───┴───┴───┴───┴───┴───┴───┘
Column-major order:
┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│ a_11 │ a_21 │ a_31 │ a_12 │ a_22 │ a_32 │ a_13 │ a_23 │ a_33 │
└───┴───┴───┴───┴───┴───┴───┴───┴───┘
```

CPU は、メモリ上の連続して並ぶデータを効率的に処理できるという性質を持つ。このため、列方向に処理を展開していきたい場合、ローメジャーのほうが高速である。もしカラムメジャーで同じ処理を行うと要素を飛び飛びに移動することになり、効率的に計算を行うことができなくなる。

NumPy の関数では、 `order` 引数で生成する配列のメモリレイアウトを指定できる。 `'C'` を指定した場合はローメジャーオーダーとなり、 `'F'` を指定した場合はカラムメジャーオーダーとなる。

個々の要素にアクセスするためにメモリ上で移動する距離をバイト数で表したものを**ストライド**（strides）と呼ぶ。

### コピーとビュー ###

配列の要素を複製した際の `numpy.ndarray` オブジェクトは、次の 2 種類に分類することができる。

  * **コピー**（copy）: もとの配列とは別にメモリを新たに確保して、もとの配列の要素の値を格納するもの
  * **ビュー**（view）: もとの配列とメモリを共有する（つまりメモリの一部または全部を参照する）もの

コピーの場合、要素の値を変更しても、もとの配列の要素は変更されない。逆に、もとの配列の要素の値を変更しても、コピーの要素は変更されない。

ビューの場合、要素の値を変更すると、もとの配列の要素の値も変わる。逆に、もとの配列の要素の値を変更すると、ビューの要素の値も変わる。

NumPy は、 配列がビューであるかどうかを調べるための関数を提供している:

``` python
numpy.shares_memory(a, b)
```

この関数は、 2 つの配列 `a` と `b` のデータ領域をしらみつぶしで調べ、配列間で共有される要素があれば `True` を返して終了する。最後まで共有される要素がなければ `False` を返す。

``` python
numpy.may_share_memory(a, b)
```

この関数は、 2 つの配列 `a` と `b` のデータ領域をメモリ境界だけ調べ、2 つの配列がメモリを共有する可能性があれば `True` を返し、そうでなければ `False` を返す。

配列がビューであるかどうかを厳密に調べるには、`numpy.shares_memory()` 関数を使う必要がある。しかし、この関数のアルゴリズムでは、判定にかかる時間が配列の要素数に対して指数関数的に増大する。

`numpy.may_share_memory()` 関数の方では、 `True` を返す場合でも配列がコピーである可能性がある。一方で `False` を返す場合、配列がコピーであることは信頼できる。実用的には、配列がビューであるのに要素を変更してもとの配列まで変更されるという事態を防ぐことができれば十分なので、 `numpy.may_share_memory()` 関数が `False` を返すことを確認すればよい。

配列生成関数
------------

### array-like オブジェクトからの生成 ###

NumPy では、配列の生成には `numpy.ndarray` のコンストラクタで直接にインスタンス化するのではなく `numpy.array()` 関数の使用が推奨されている。実際、 `numpy.ndarray` の `__repr__()` は `numpy.array()` 関数を使った式を見やすいように改行した形で出力する。この出力は `numpy.array` というクラスのインスタンスを意味するわけではないことに注意する。

`numpy.array()` 関数の引数は次のように定義されている。

``` python
numpy.array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0, like=None)
```

この関数は、第1引数に指定した array-like オブジェクトから配列（`numpy.ndarray` のインスタンス）を生成して返す。

**array-like オブジェクト**:

  * 配列（`numpy.ndarray` オブジェクト）
  * `array.array` オブジェクト
  * Python の（ネストされた）シーケンス──多重リストや多重タプルなど
  * Python のスカラー
  * `__array__()` メソッドが配列を返すオブジェクト

主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `object` | 配列の内容を与える array-like オブジェクトを指定する |
| `dtype` | 配列の要素の型 dtype を指定する。指定されない場合、NumPy は値を表すことができるデフォルトの dtype を使用しようとする |
| `copy` | `True`（デフォルト）の場合、`object` が配列ならコピーを作成する。`False` の場合、`object` が配列ならビューを作成する |
| `order` | 配列のメモリレイアウトを指定する。以下の文字列を指定できる<br /><br />・`'C'`: ローメジャーオーダー<br /><br />・`'F'`: カラムメジャーオーダー<br /><br />・`'K'`: `object` が配列の場合、そのメモリレイアウトが維持される。それ以外の場合は類似したオーダー（ほとんどの場合でローメジャーオーダー）になる。デフォルト<br /><br />・`'A'`: `object` がカラムメジャーオーダーなら維持されるが、それ以外の場合は常にローメジャーオーダーになる |
| `subok` | `False`（デフォルト）の場合、`object` が `numpy.ndarray` のサブクラスのインスタンスであっても常に `numpy.ndarray` インスタンスを返す |
| `ndmin` | 生成する配列の最小次元数を指定する。この要件を満たすために、必要に応じて先頭に軸が追加されていく。`ndmin` が `0`（デフォルト）の場合、軸は追加されない |

次のコードは、第 1 引数に整数のリストを渡して呼び出すことにより 1 次元配列を得る例である。

In [63]:
# 1次元配列 -- 長さ 3 のベクトル
np.array([1, 2, 3])

array([1, 2, 3])

第 1 引数にネストされたシーケンスを渡す場合は、各シーケンスの長さが揃っていないと `ValueError` 例外が発生する。

In [ ]:
# 2次元配列 -- 2x3 の行列
np.array([[0., 0.1, 0.2], [1., 1.1, 1.2]])

array([[0. , 0.1, 0.2],
       [1. , 1.1, 1.2]])

In [ ]:
# 3次元配列 -- テンソル
np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], dtype=float)  # dtype 引数で numpy.float64 型を指定する

array([[[1., 2.],
        [3., 4.]],

       [[5., 6.],
        [7., 8.]]])

`numpy.array()` に既存の配列を渡すと、そのコピーを返すことに注意する。ただし、`copy` 引数を `False` に指定した場合は配列のビューを返す。

In [ ]:
a = np.array([1, 2, 3])
a_copy = np.array(a)
a_view = np.array(a, copy=False)
assert not np.shares_memory(a, a_copy)
assert np.shares_memory(a, a_view)

``` python
numpy.copy(a, order='K', subok=False)
ndarray.copy(order='C')
```

`numpy.copy(a)` は `numpy.array(a, copy=True)` と等価である。この関数は、明示的に既存の配列のコピーを作成するために使用する。また、配列は `copy()` メソッドも持ち、 `a.copy()` は `numpy.copy(a, order='C', subok=True)` と等価である。

In [ ]:
a = np.array([1, 2, 3])
a_view = np.array(a, copy=False)  # ビューを作成
a_copy = np.copy(a_view)
assert not np.shares_memory(a, a_copy)

### 初期化された配列の生成 ###

``` python
numpy.zeros(shape, dtype=float, order='C', *, like=None)
```

この関数は、全要素を `0` で初期化した配列を生成する。

| 引数 | 意味 |
|:---|:---|
| `shape` | 配列の形状を整数のタプルで指定する。 1 次元の場合は `shape` に単独の整数を指定できる |
| `dtype` | 配列の要素の型 dtype を指定する。指定されない場合は `numpy.float64` が使われる |
| `order` | 配列のメモリレイアウトを指定する。`'C'` か `'F'` を指定できる。デフォルトは `'C'` |

In [ ]:
np.zeros(3)  # 長さ 3 の零ベクトル

array([0., 0., 0.])

``` python
numpy.ones(shape, dtype=None, order='C', *, like=None)
```

この関数は、全要素を `1` で初期化した配列を生成する。引数の意味は `numpy.zeros()` 関数と同じ。

In [ ]:
np.ones((2, 3), int)  # 2x3 の行列

array([[1, 1, 1],
       [1, 1, 1]])

``` python
numpy.full(shape, fill_value, dtype=None, order='C', *, like=None)
```

この関数は、全要素を `fill_value` で初期化した配列を生成する。他の引数の意味は `numpy.zeros()` 関数の同名の引数と同じ。ただし、 `dtype` 引数を指定しない場合、 NumPy は値を表すことができるデフォルトの dtype を使用しようとする。

In [ ]:
np.full((2, 2), 10)

array([[10, 10],
       [10, 10]])

`fill_value` には array-like オブジェクトを指定できる。

In [ ]:
np.full((2, 2), [1, 2])

array([[1, 2],
       [1, 2]])

``` python
numpy.zeros_like(a, dtype=None, order='K', subok=True, shape=None)
```

この関数は、第 1 引数 `a` に指定した array-like オブジェクトと同じ形状と型で要素が `0` の配列を生成する。 `numpy.array()` 関数と同様に `dtype`、`order`、`shape` 引数を受け入れ、それぞれ型、メモリレイアウト、形状を変更することができる。

In [ ]:
a = np.array([[1, 2, 3], [2, 3, 4]])
np.zeros_like(a)

array([[0, 0, 0],
       [0, 0, 0]])

``` python
numpy.ones_like(a, dtype=None, order='K', subok=True, shape=None)
```

この関数は、第 1 引数 `a` に指定した array-like オブジェクトと同じ形状と型で要素が `1` の配列を生成する。引数の意味は `numpy.zeros_like()` 関数と同じ。

In [ ]:
a = np.array([[1, 2, 3], [2, 3, 4]])
np.ones_like(a, np.float64)

array([[1., 1., 1.],
       [1., 1., 1.]])

``` python
numpy.full_like(a, fill_value, dtype=None, order='K', subok=True, shape=None)
```

この関数は、第 1 引数 `a` に指定した array-like オブジェクトと同じ形状と型で要素が `fill_value` の配列を生成する。 `fill_value` に array-like オブジェクトを指定することができる。その他の引数の意味は `numpy.zeros_like()` 関数と同じ。

In [ ]:
a = np.zeros(3)
np.full_like(a, 255, 'u1')

array([255, 255, 255], dtype=uint8)

### 初期化されていない配列の生成 ###

``` python
numpy.empty(shape, dtype=float, order='C', *, like=None)
```

この関数は、初期化されていない配列を生成する。引数の意味は `numpy.zeros()` 関数と同じ。

関数の名前は `empty`（空）であるが、一般に多次元配列が空配列であるとは、ある次元の長さが `0` であることを指すのであって、 `numpy.empty()` は空配列を生成するための関数というわけではない（たとえば `numpy.empty(0)` や `numpy.zeros(0)` のようにすれば全ての次元で長さが `0` の空配列を生成できる）。

初期化されていない配列では、配列の要素の値を設定しないため、配列の生成がわずかに高速になる。ただし、確保されたメモリの値がそのまま読み込まれるので、配列の要素の値は不定となることに注意する。

In [ ]:
np.empty((2, 2))

array([[5.05316266e-310, 0.00000000e+000],
       [6.63203708e-310, 6.63207187e-310]])

``` python
numpy.empty_like(prototype, dtype=None, order='K', subok=True, shape=None)
```

この関数は、初期化されていない配列を生成することを除いて、`numpy.zeros_like()` と同様である。

In [ ]:
a = ([1, 2, 3], [4, 5, 6])
np.empty_like(a)

array([[4607182418800017408, 4607182418800017408, 4607182418800017408],
       [4607182418800017408, 4607182418800017408, 4607182418800017408]])

### 特別な配列の生成 ###

NumPy は、特別なベクトルや行列を表す配列を生成する関数も提供している。主な関数は以下の通り。

  * 1 次元配列（ベクトル）
      * `numpy.arange()`
      * `numpy.linspace()`
  * 2 次元配列（行列）
      * `numpy.eye()`
      * `numpy.identity()`
      * `numpy.tri()`
      * `numpy.tril()`
      * `numpy.triu()`
      * `numpy.diag()`

``` python
numpy.arange([start, ]stop, [step, ]dtype=None, *, like=None)
```

この関数は、 Python 組み込みの `range()` の配列版で、等差数列を表す 1 次元配列を生成する。 `start`、`stop`、`step` 引数の意味は、 `range()` の同名の引数と同じ。ただし、`range()` と異なり、浮動小数点数 `float` を引数に指定できる。 dtype は、 `start`、`stop`、`step` 引数から自動的に選択されるが、 `dtype` 引数で指定することもできる。

In [ ]:
np.arange(3)

array([0, 1, 2])

In [ ]:
np.arange(0., 1., .2)

array([0. , 0.2, 0.4, 0.6, 0.8])

``` python
numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None, axis=0)
```

この関数も等差数列を表す 1 次元配列を生成するが、間隔（公差）ではなく要素数を指定する。

| 引数 | 意味 |
|:---|:---|
| `start` | 最初の値 |
| `stop` | 最後の値 |
| `num` | 生成する配列の要素数。デフォルトでは `50` |
| `endpoint` | `True`（デフォルト）の場合、生成する配列の要素に `stop` が含まれる。`False` の場合、 `stop` が含まれない |
| `retstep` | `True` の場合、この関数は生成した配列とその間隔の 2 要素タプル `(samples, step)` を返す |
| `dtype` | 配列の dtype。指定しない場合は `start` と `stop` から自動的に選択されるが、整数型が選択されることはない |
| `axis` | この引数は `start` または `stop` がスカラーでない array-like オブジェクトの場合にのみ意味を持つ。生成された結果をこの引数で指定した軸に沿って配置する。デフォルトでは<br /> `0` 軸。`-1` を指定すると末尾の軸に沿って配置する |

この関数によって生成される配列の間隔は、 `start`、`stop`、`num` に応じて計算される。

In [ ]:
np.linspace(1, 10, 10)  # start、stop、num が整数を生成するような場合でも dtype は整数型とはならない

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

`start > stop` であってもよい。

In [ ]:
np.linspace(1., 0., 5, endpoint=False)

array([1. , 0.8, 0.6, 0.4, 0.2])

計算された間隔を取得したい場合は、 `retstep` 引数に `True` を渡す。このとき、関数の戻り値は 2 要素タプルとなり、第 1 要素に生成された配列が、第 2 要素に計算された間隔が入っている。

In [ ]:
np.linspace(1., 0., 5, endpoint=False, retstep=True)

(array([1. , 0.8, 0.6, 0.4, 0.2]), -0.2)

``` python
numpy.eye(N, M=None, k=0, dtype=<class 'float'>, order='C', *, like=None)
```

この関数は、`1` が斜めに並んで、それ以外は `0` となる 2 次元配列を生成する。

| 引数 | 意味 |
|:---|:---|
| `N` | 行のサイズ |
| `M` | 列のサイズ。`None`（デフォルト）の場合、 `N` が使われる |
| `k` | `1` の始まり位置。デフォルトは `0`（先頭） |
| `dtype` | dtype。デフォルトは `numpy.float64` |
| `order` | 配列のメモリレイアウト。`'C'` か `'F'` を指定できる。デフォルトは `'C'` |

デフォルトでは `N` 次単位行列を表す 2 次元配列を生成する。 `M` 引数を指定することで、正方行列ではない行列として配列を作成することもできる。

In [ ]:
np.eye(3, 4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

`k` 引数で `1` の斜めの並びを左右にシフトすることができる。正の整数で列方向にシフトさせ、負の整数で列方向の反対方向にシフトさせる。

In [ ]:
np.eye(3, 4,k=1)

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [ ]:

np.eye(3, 4, k=-1)

array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

``` python
numpy.identity(n, dtype=None, *, like=None)
```

この関数は、 `numpy.eye(n)` と同じであり、明示的に `n` 次単位行列としての配列を生成する。第 2 引数 `dtype` で dtype を指定することもできる。

In [ ]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

``` python
numpy.tri(N, M=None, k=0, dtype=<class 'float'>, *, like=None)
```

この関数は、対角線より上の成分が `0`、それ以外の成分が `1` の下三角行列を表す 2 次元配列を生成する。引数の意味は `numpy.eye()` 関数と同じ。ただし、`order` は指定できない。

In [ ]:
np.tri(3, 5, 2, dtype=int)

array([[1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

``` python
numpy.tril(m, k=0)
numpy.triu(m, k=0)
```

`numpy.tril()` 関数と `numpy.triu()` 関数は、第 1 引数に指定した array-like オブジェクト `m` から、それぞれ下三角行列と上三角行列を抽出した形の 2 次元配列を生成する。 `m` は正方行列でなくてもよい。第 2 引数 `k` の意味は、 `numpy.eye()` 関数の `k` 引数と同じ。

In [ ]:
np.tril([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

array([[ 1,  0,  0],
       [ 4,  5,  0],
       [ 7,  8,  9],
       [10, 11, 12]])

In [ ]:
np.triu([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], -1)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 0,  8,  9],
       [ 0,  0, 12]])

``` python
numpy.diag(v, k=0)
```

この関数は、 array-like オブジェクト `v` が 2 次元配列の場合はその対角成分を抽出し、 1 次元配列の場合はその要素を対角成分とする対角行列を生成する。第 2 引数 `k` の意味は、 `numpy.eye()` 関数の `k` 引数と同じ。

In [ ]:
np.diag([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

array([1, 5, 9])

In [ ]:
np.diag([1, 5, 9])

array([[1, 0, 0],
       [0, 5, 0],
       [0, 0, 9]])

### 乱数の生成 ###

NumPy では、独自に乱数（正確には疑似乱数）の生成をサポートする `numpy.random` モジュールが利用できる。

``` python
numpy.random.default_rng(seed=None)
```

この関数は、乱数生成器のクラス `numpy.random.Generator` のインスタンスを生成する。実は、NumPy では乱数生成器を選ぶことができ、この関数は乱数生成器として [PCG64](https://ja.wikipedia.org/wiki/Permuted_congruential_generator) を使用する。他の乱数生成器を選ぶメリットはないので、 `numpy.random.default_rng()` 関数で十分。

唯一の引数 `seed` に乱数の種となる `int` 型の値または `int` 型の array-like オブジェクトを指定する。 `seed` の指定を省略した場合、 `numpy.random.default_rng()` 関数は OS の機能を利用して予測不可能な乱数の種を取得する。

`numpy.random.Generator` インスタンスの主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `Generator.random(size=None, dtype=np.float64,`<br />` out=None)` | `0.0` 以上 `1.0` 未満の範囲でランダムな浮動小数点数を取得する。戻り値は `size` の指定によって異なる。<br /><br />・`size` が整数のタプルの場合: その形状の配列が返される<br /><br />・`size` が整数の場合: その長さの 1 次元配列が返される<br /><br />・`size` が `None`（デフォルト）の場合: 単一の値が返される<br /><br />`out` に配列を指定した場合、その配列の要素に抽出した値がセットされる。`out` の形状は `size` と一致する必要がある | 配列 or<br />単一の値 |
| `Generator.uniform(low=0.0, high=1.0, `<br />`size=None)` | `low` 以上 `high` 未満の範囲でランダムな浮動小数点数を取得する。`size` の意味は `random()` の同名の引数と同じ | 配列 or<br />単一の値 |
| `Generator.integers(low, high=None, size=None,`<br />` dtype=np.int64, endpoint=False)` | ランダムな整数を取得する。範囲は、第 2 引数 `high` を省略した場合に `0` 以上 `low` 未満、そうでない場合に `low` 以上 `high` 未<br />満となる。ただし、`endpoint` 引数が `True` の場合、両端点が範囲に含まれる。`size` の意味は `random()` の同名の引数と同じ | 配列 or<br />単一の値 |
| `Generator.choice(a, size=None, replace=True,`<br />` p=None, axis=0, shuffle=True)` | `a` が配列の場合、`a` からランダムなサンプルを生成する。`a` が整数の場合、`np.arange(a)` からランダムなサンプルを生成する。<br />`size` の意味は `random()` の同名の引数と同じ。`replace` が `True`（デフォルト）の場合、要素の重複選択を許可する。`p` に 1 次配<br />列を指定した場合、それは `a` の各要素に関連付けられた確率を意味する。`p` を指定しない場合、`a` の各要素は均一な確率を仮<br />定する | 配列 or<br />単一の値 |

In [ ]:
rng = np.random.default_rng()
print(rng.random())
print(rng.integers(10, size=10))
a = np.empty((2, 3))
rng.random((2, 3), out=a)
print(a)
print(rng.choice(5, 3, p=[0.1, 0, 0.3, 0.6, 0]))

0.5375432908520157
[7 1 8 8 7 6 7 3 3 8]
[[0.32909559 0.94418078 0.04301196]
 [0.71725376 0.24699596 0.37808083]]
[3 3 3]


【NumPy 2.0 での変更点】  
以下の関数では、 `device` キーワード専用引数が追加されている。

  * `numpy.ones()`
  * `numpy.full()`
  * `numpy.zeros_like()`
  * `numpy.ones_like()`
  * `numpy.full_like()`
  * `numpy.empty()`
  * `numpy.empty_like()`
  * `numpy.arange()`
  * `numpy.linspace()`
  * `numpy.eye()`

配列の属性
----------

`numpy.ndarray` の属性は、基本的に読み取り専用プロパティとして実装されている。

### 型・次元数・形状・サイズ・全要素 ###

配列の型、次元数、形状、サイズ（全要素数）、全要素を取得するには、 `numpy.ndarray` の以下の属性を使う。

| 属性 | 意味 |
|:---|:---|
| `dtype` | 配列の dtype |
| `ndim` | 配列の次元数 |
| `shape` | 配列の形状を表すタプル。タプルのサイズは `ndim` と一致する。タプルには次元の小さい順（つまり軸の番号の小さい順）で各配列のサイズが並ぶ |
| `size` | 配列に含まれる要素の数。`shape` の中の整数を全て乗じた数と一致する |
| `flat` | 配列の要素を 1 つずつ返すイテレーターを返す |

In [ ]:
# 1 次元配列の場合
a1 = np.arange(3)
print(f"{a1!r}")
print(f"{a1.dtype=}, {a1.ndim=}, {a1.shape=}, {a1.size=}")

print("-" * 66)

# 2 次元配列の場合
a2 = np.array([a1, a1])
print(f"{a2!r}")
print(f"{a2.dtype=}, {a2.ndim=}, {a2.shape=}, {a2.size=}")

print("-" * 66)

# 3 次元配列の場合
a3 = np.array([a2, a2])
print(f"{a3!r}")
print(f"{a3.dtype=}, {a3.ndim=}, {a3.shape=}, {a3.size=}")

array([0, 1, 2])
a1.dtype=dtype('int64'), a1.ndim=1, a1.shape=(3,), a1.size=3
------------------------------------------------------------------
array([[0, 1, 2],
       [0, 1, 2]])
a2.dtype=dtype('int64'), a2.ndim=2, a2.shape=(2, 3), a2.size=6
------------------------------------------------------------------
array([[[0, 1, 2],
        [0, 1, 2]],

       [[0, 1, 2],
        [0, 1, 2]]])
a3.dtype=dtype('int64'), a3.ndim=3, a3.shape=(2, 2, 3), a3.size=12


`flat` 属性が返すイテレーターは、軸の番号の低い順に辞書式順序で要素を返す。たとえば、2 次元配列の場合、まず最初の行の要素を順番に返し、次に 2 行目の行の要素を順番に返す、というようになる。

In [ ]:
a = np.array([[1, 2], [3, 4]])
print(f"{a!r}")
for x in a.flat:
    print(x)

array([[1, 2],
       [3, 4]])
1
2
3
4


### メモリ情報 ###

配列のメモリに関する情報を取得するには、 `numpy.ndarray` の以下の属性を使う。

| 属性 | 意味 |
|:---|:---|
| `base` | 配列がビューである場合は、もととなる配列を指す。そうでない場合は `None` |
| `flags` | 配列のメモリレイアウトに関する情報を格納するオブジェクトを指す。各情報には `a.flags['KEY']` のように大文字のキー名で辞書のようにアクセスするか、`a.flags.key` のように<br />小文字の属性名で属性参照の形でアクセスできる。主なキーは以下の通り<br /><br />・`C_CONTIGUOUS`: ローメジャーオーダーなら `True`<br /><br />・`F_CONTIGUOUS`: カラムメジャーオーダーなら `True`<br /><br />・`WRITEABLE`: データ領域への書き込みが可能なら `True` |
| `itemsize` | バイト単位での各要素のメモリ消費量 |
| `strides` | 各次元方向に 1 つ隣の要素に移動するために必要なバイト数をタプルで表したもの |
| `nbytes` | バイト単位での配列のデータ領域のメモリ消費量。`itemsize` と `size` の積に一致する |

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]], dtype=float)
assert a.base is None
assert a.flags['C_CONTIGUOUS'] and a.flags.writeable
print(f'{a.size=}')
print(f'{a.itemsize=}')  # numpy.float64 なので 8 バイト（＝ 64 ビット）
print(f'{a.strides=}')  # ローメジャーオーダーなので、行方向へは 8x3(=24)、列方向は 8
print(f'{a.nbytes=}')  # 8x6=48 バイト

a.size=6
a.itemsize=8
a.strides=(24, 8)
a.nbytes=48


配列のインデックス参照
----------------------

### 基本的なインデックス ###

配列を構成する低次元の配列には、 Python の多重リストと同様に低い次元からインデックス参照を重ねる方法 `[i][j][k]...` で参照できる。配列内の個々の要素を参照する方法も同様である。

In [ ]:
a= np.array([[[1, 2],
              [3, 4]],
             [[5, 6],
              [7, 8]]])
print(f"{a[1][0]=}")
assert a[0][1][1] == 4

a[1][0]=array([5, 6])


しかし、 2 次元以上の配列では、この参照方法はあまり使われない。配列のインデックス参照が呼び出す `__getitem__()` メソッドは C の関数で実装され、単一の整数やスライスだけでなく、タプルも受け付ける。タプルを渡す場合、低い次元から順にインデックスを並べた形式のタプルを渡すようにする。 Python では、カンマで区切った式のリストがタプル評価されるから、`()` を省略して `[i, j, k, ...]` のように各次元のインデックスをカンマ区切りで指定することができる。この独自の参照方法がよく使われる。

In [ ]:
a= np.array([[[1, 2],
              [3, 4]],
             [[5, 6],
              [7, 8]]])
print(f"{a[1, 0]=}")
assert a[0, 1, 1] == 4

a[1, 0]=array([5, 6])


独自の参照方法でも、負の値を使うことや、スライス `start:stop` や `start:stop:step` を使うことができる。スライスの省略記法も Python のシーケンスと同様である。 2 次元配列の場合、`[i]` と `[i, :]` は等価で `i` 番目の行の全体を指定し、また、 `[:, j]` は `j` 番目の列の全体を指定する。

In [ ]:
a = np.array([[0, 1, 2, 3, 4],
              [5, 6, 7, 8, 9]])
assert a[-1, -2] == 8  # 負の数の場合、インデックスは配列の末尾からの相対インデックスになる
print(f"   [0, 1:4]: {a[0, 1:4]!r}")  # 0 行目の 1 番目から 3 番目までのスライス
print(f"    [0, :4]: {a[0, :4]!r}")  # 0 行目の 0 番目から 3 番目までのスライス
print(f"    [0, 1:]: {a[0, 1:]!r}")  # 0 行目の 1 番目から 4 番目までのスライス
print(f"[1, 1:-1:2]: {a[1, 1:-1:2]!r}")  # 1 行目の 1 番目から 3 番目まで、2 ごとのスライス
print(f"   [1, ::2]: {a[1, ::2]!r}")  # 1 行目の 2 ごとのスライス
print(f"     [1, :]: {a[1, :]!r}")  # 1 行目のスライス（[1] と等価）

   [0, 1:4]: array([1, 2, 3])
    [0, :4]: array([0, 1, 2, 3])
    [0, 1:]: array([1, 2, 3, 4])
[1, 1:-1:2]: array([6, 8])
   [1, ::2]: array([5, 7, 9])
     [1, :]: array([5, 6, 7, 8, 9])


In [ ]:
a = np.array([[[1, 2],
               [3, 4]],
              [[5, 6],
               [7, 8]]])
a[:, :, 0]  # 最後の次元だけ指定（行列ごとに各行の 0 番目の列を取得）

array([[1, 3],
       [5, 7]])

`a[:, :, 0]` のように複数の `:` を連続して指定する場合、それらをまとめて 1 つの `Ellipsis`（`...` と同じ）で表す省略記法が使える。

  * `[..., i]`: 最大の次元のみを指定
  * `[i, ..., j]`: 最小の次元と最大の次元のみを指定
  * `[...]`: 配列の全体を指定

`:` を続けて書く場合には `:` の数が次元数に一致しないとエラーになるが、 `...` の省略記法を使う場合にはそのようなエラーを防ぐことができる。なお、 `[i, ...]` は `[i]` と等価となる。

`[..., i, ..., j]` のように 1 回のインデックス参照に `...` を 2 つ以上使うと、インデックスの指定が不明確になるのでエラーが発生する。

In [ ]:
a = np.array([[[1, 2],
               [3, 4]],
              [[5, 6],
               [7, 8]]])
a[..., 0]  # a[;, :, 0] と同じ

array([[1, 3],
       [5, 7]])

Python のシーケンスではスライスを使用する場合にコピーが返されるが、 **`numpy.ndarray` オブジェクトではスライスを使用する場合にビューが返される**。`Ellipsis` を使用する場合も同様である。

### 次元を追加するインデックス ###

配列のインデックス参照では、`None` を使うと、その位置に次元（軸）を 1 つ追加した新しい配列が返される。`None` のこの用途では、わかりやすい別名の `numpy.newaxis` が使われる。

追加された次元では、サイズが 1 とされる。もとの配列内の配列や要素は、追加された次元でサイズが 1 となるように分配される（最初からサイズが 1 となる場合には分配は行われない）。

**次元を追加するインデックスを使用する場合、ビューが返される**。

In [ ]:
# 1 次元配列に新しい軸として 0 軸を追加する例
a = np.arange(5)
a[np.newaxis, :]  # 1x5 行列となり、分配は行われない

array([[0, 1, 2, 3, 4]])

In [ ]:
# 1 次元配列に新しい軸として 1 軸を追加する例
a = np.arange(5)
a[:, np.newaxis]  # 5x1 行列となり、分配が行われる

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [ ]:
# 2 次元配列に新しい軸として 1 軸を追加する例
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
a[:, np.newaxis, :]  # 3 つの 1x3 行列からなる 3 次元配列となり、分配が行われる

array([[[1, 2, 3]],

       [[4, 5, 6]],

       [[7, 8, 9]]])

### ブーリアンインデックス ###

配列のインデックス参照では、各次元でのインデックスを指定するために、整数やスライスに代えて、その次元のサイズ（要素数）と同じサイズのブール値のリストや配列を指定することができる。この場合、 `True` の位置にある要素のみを抽出した新しい配列が返される。このようなインデックスを**ブーリアンインデックス**（boolean index）と呼ぶ。

ブーリアンインデックスは、位置を柔軟に指定できるように強化したスライスのようであるが、以下の点に注意する。

  * ブーリアンインデックスのサイズが次元のサイズと一致していなければ、 `IndexError` 例外が発生する（スライスではエラーが発生しないこととは対照的である）。
  * **ブーリアンインデックスを使用する場合、ビューではなくコピーが返される**。

1 次元配列でブーリアンインデックスを使用する場合、スライスを使用する場合と同様に、配列（ただし配列のコピー）が返されるのであって、単一の要素を参照することはできない。

In [ ]:
# 1 次元配列の例
a = np.arange(4)
# [0, 1, 2, 3]
a[[False, True, False, False]], a[[True, False, True, False]]

(array([1]), array([0, 2]))

2 次元以上の配列では、全ての次元でブーリアンインデックスを使う必要はなく、整数やスライスを使うインデックスと組み合わせて使用することができる。

In [ ]:
# 2 次元配列で行にブーリアンインデックスを使用する例
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
a[[True, False, True]]

array([[1, 2, 3],
       [7, 8, 9]])

In [ ]:
# 2 次元配列で、行にブーリアンインデックス、列に整数インデックスを使用する例
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
a[[True, False, True], 2]

array([3, 9])

In [ ]:
# 2 次元配列で、行にスライス、列にブーリアンインデックスを使用する例
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
a[:, [True, False, True]]

array([[1, 3],
       [4, 6],
       [7, 9]])

ブロードキャスト
----------------

### ブロードキャストの基本 ###

NumPy では、配列に対して、次元数と形状をあるサイズに揃えるような変更が以下の 2 規則に従って行われることを**ブロードキャスト**（broadcasting）と呼んでいる。

  1. 次元数を揃える  
配列の次元数が小さい場合は、先頭に軸が追加され、その次元ではサイズが 1 の配列が含まれる形にする（つまり `a[numpy.newaxis, :]` の形にする）。
  2. 各次元の大きさを揃える  
ある次元の配列のサイズが小さい場合は、そのサイズが 1 であるときに限って、値を繰り返す形で配列を引き延ばす。

上記の 2 規則に従って配列を変換することができるとき、配列は**ブロードキャスト可能**（broadcastable）と呼ばれる。ブロードキャストで行われる操作は上記の 2 規則で全てとなるので、以下のような配列では全てブロードキャストが失敗する。

  * 入力配列の次元数が大きい場合
  * ある次元で配列のサイズが大きい場合
  * ある次元で配列のサイズが小さく、かつ、そのサイズが 2 以上である場合

一方、全てのスカラーは、ブロードキャストの文脈では 0 次元配列として扱われ、常にブロードキャスト可能である。

配列にブロードキャストが行われると、読み取り専用ビューが作成される。一般の配列のメモリレイアウトと異なり、通常は連続していない。さらに、その複数の要素は単一のメモリ位置を参照する場合がある。

ブロードキャストは、配列を扱う様々な場面で NumPy によって暗黙的に行われ、プログラマーが明示的に行うようなものではない。しかしながら、ブロードキャストの結果を確認したい場合もあるので、明示的にブロードキャストを行う関数が提供されている。

``` python
numpy.broadcast_to(array, shape, subok=False)
```

この関数は、 array-like オブジェクト `array` を、指定された形状 `shape` にブロードキャストする。

次のコードは、スカラーを形状が `(3,)` の 1 次元配列にブロードキャストする。

In [ ]:
# スカラーを 1 次元配列へ
np.broadcast_to(1, (3,))  # np.full((3,), 1) と同じ

array([1, 1, 1])

このブロードキャストでは、次の手順で操作が行われる。

``` text
1. 入力:
    1            【形状 ()】
2. 次元数を 1 に揃える:
    [1]          【形状 (1,)】
3. 0 軸での配列のサイズを 3 に揃える:
    [1, 1, 1]    【形状 (3,)】
```

次のコードは、スカラーを形状が `(2, 3)` の 2 次元配列にブロードキャストする。

In [ ]:
# スカラーを 2 次元配列へ
np.broadcast_to(1, (2, 3))  # np.full((2, 3), 1) と同じ

array([[1, 1, 1],
       [1, 1, 1]])

このブロードキャストでは、次の手順で操作が行われる。

``` text
1. 入力:
    1              【形状 ()】
2. 次元数を 2 に揃える:
    [[1]]          【形状 (1, 1)】
3. 0 軸での配列のサイズを 2 に揃える:
    [[1],          【形状 (2, 1)】
     [1]]
4. 行ごとに 1 軸での配列のサイズを 3 に揃える:
    [[1, 1, 1],    【形状 (2, 3)】
     [1, 1, 1]]
```

次のコードは、サイズが 3 であるリスト（1 次元配列に相当する array-like オブジェクト）を形状が `(3, 3)` の 2 次元配列にブロードキャストする。

In [ ]:
np.broadcast_to([1, 2, 3], (3, 3))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

このブロードキャストでは、次の手順で操作が行われる。

``` text
1. 入力:
    [1, 2, 3]      【形状 (3,)】
2. 次元数を 2 に揃える:
    [[1, 2, 3]]    【形状 (1, 3)】
3. 0 軸での配列のサイズを 3 に揃える:
    [[1, 2, 3],    【形状 (3, 3)】
     [1, 2, 3],
     [1, 2, 3]]
```

次のコードは、形状が `(3, 1)` である 2 次元配列を形状が `(3, 3)` の 2 次元配列にブロードキャストする。

In [ ]:
a = np.arange(1, 4)[:, np.newaxis]
# [[1],
#  [2],
#  [3]]
np.broadcast_to(a, (3, 3))

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

このブロードキャストでは、次元数と 0 軸での配列のサイズは一致しているので操作する必要がない。次の手順で操作が行われる。

``` text
1. 入力:
    [[1],          【形状 (3, 1)】
     [2],
     [3]]
2. 行ごとに 1 軸での配列のサイズを 3 に揃える:
    [[1, 1, 1],    【形状 (3, 3)】
     [2, 2, 2],
     [3, 3, 3]]
```

次のコードは、形状が `(2, 2)` である 2 次元配列を形状が `(2, 2, 2)` の 3 次元配列にブロードキャストする。 0 軸が挿入されて行列が繰り返されるブロードキャストが行われることがわかる。

In [ ]:
a = np.array([[1, 2],
              [3, 4]])
np.broadcast_to(a, (2, 2, 2))

array([[[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]]])

``` python
numpy.broadcast_arrays(*args, subok=False)
```

この関数は、複数の array-like オブジェクトを入力として、ブロードキャストして形状を揃えた配列のリストを返す。具体的には、まず次元数が入力配列の中で最大のものに揃えられ、次に 0 軸から順に配列のサイズが次元数を揃えた配列の中で最大のものに揃えられる。

次のコードは、スカラーと、形状が `(2,)` の 1 次元配列に相当するリストと、形状が `(2, 1)` の 2 次元配列に相当するネストされたリストに対して、それぞれ形状が `(2, 2)` の 2 次元配列にブロードキャストされた配列を生成する。

In [ ]:
np.broadcast_arrays(1, [2, 3], [[4], [5]])

[array([[1, 1],
        [1, 1]]),
 array([[2, 3],
        [2, 3]]),
 array([[4, 4],
        [5, 5]])]

このブロードキャストでは、次の手順で操作が行われる。

``` text
1. 入力:
    1,  [2, 3],  [[4],                 【形状 () (2,) (2, 1)】
                  [5]]
2. 3 つ目の配列の次元数が採用され、次元数を 2 に揃える:
    [[1]],  [[2, 3]],  [[4],           【形状 (1, 1) (1, 2) (2, 1)】
                       [5]]
3. 3 つ目の配列の 0 軸での配列のサイズが採用され、0 軸での配列のサイズを 2 に揃える:
    [[1],   [[2, 3],    [[4],          【形状 (2, 1) (2, 2) (2, 1)】
     [1]]    [2, 3]]     [5]]
4. 2 つ目の配列の 1 軸での配列のサイズが採用され、1 軸での配列のサイズを 2 に揃える:
    [[1, 1],   [[2, 3],    [[4, 4],    【形状 (2, 2) (2, 2) (2, 2)】
     [1, 1]]    [2, 3]]     [5, 5]]
```

### ファンシーインデックス ###

配列のインデックス参照を実装する C の関数は、必要により入力されたインデックスを整数配列に変換し、それらの形状と配置を使って新しい配列を構築する。入力がタプルの場合、整数配列への変換に際し、タプルの全てのエントリは暗黙的にブロードキャストされて形状が揃ったものとなる。ブロードキャスト可能でない場合は、`IndexError` 例外が発生する。

例えば、形状が `(2, 5)` の 2 次元配列で、整数インデックスとスライスを組み合わせた `a[0, 1:4]` というインデックス参照を考える。 0 軸のインデックス値 `0` はブロードキャストの結果 `[0, 0, 0]` に変換され、 1 軸のインデックス値であるスライス `1:4` は `[1, 2, 3]` に変換される。このインデックス参照から返される配列（ビュー）は 1 次元配列とされ、 `a[0, 1], a[0, 2], a[0, 3]` を含むものとなる。

ブーリアンインデックスは、 `True` の位置（整数）からなる配列に変換される。例えば、形状が `(3, 3)` の 2 次元配列で、スライスとブーリアンインデックスを組み合わせた `a[:, [True, False, True]]` というインデックス参照を考える。 0 軸のインデックス値であるスライス `:` は `[[0], [1], [2]]` に変換され、 1 軸のブーリアンインデックス `[True, False, True]` は `[0, 2]` に変換され、次のようにブロードキャストされる:

``` text
[[0],    [0, 2]                              [[0, 0],    [[0, 2],
 [1],             ─（ブロードキャスト）→    [1, 1],     [0, 2],
 [2]],                                        [2, 2]],    [0, 2]]
```

このインデックス参照から返される配列（コピー）は 2 次元配列とされ、次のような配列が返される:

``` text
[a[0, 0], a[0, 2],
 a[1, 0], a[1, 2],
 a[2, 0], a[2, 2]]
```

このように、インデックスとして指定した整数、スライス、ブーリアンインデックスは整数配列に変換されるのであるが、はじめからインデックスに整数のリストや配列を指定することもできる。このようなインデックスを**ファンシーインデックス**（fancy index）という。

**ファンシーインデックスを使用する場合、ビューではなくコピーが返される**。

In [ ]:
a = np.array([[0, 1, 2, 3, 4],
              [5, 6, 7, 8, 9]])
a[0, 1:4], a[[0, 0, 0], [1, 2, 3]]  # 各要素が同じである配列が返される

(array([1, 2, 3]), array([1, 2, 3]))

In [ ]:
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
(a[:, [True, False, True]],
 a[[[0],
    [1],
    [2]], [0, 2]])  # 各要素が同じである配列が返される

(array([[1, 3],
        [4, 6],
        [7, 9]]),
 array([[1, 3],
        [4, 6],
        [7, 9]]))

ファンシーインデックスでは、インデックスを自由に指定することができる。これにより要素の並べ替えができる。**インデックスを重複して指定してもよい**。インデックスに負の値も使用できる。

インデックス参照で返される配列の次元数と形状は、ブロードキャスト後の整数配列の次元数と形状に一致するから、ファンシーインデックスを使えば次元数と形状を自由に操作することができる。例えば、インデックス参照で形状が `(N, M)` の 2 次元配列を取得したいなら、（ブロードキャストされて）形状が `(N, M)` の 2 次元配列となるようなファンシーインデックスを指定すればよく、もとの配列の形状には縛られない。これにより、もとの配列よりも要素数の多い配列を構築することもできる。

In [ ]:
# 1 次元配列の例
a = np.arange(1, 4)
(a,
 a[[0, 2]],
 a[[2, 1, 0]],
 a[[-1, 2, 2, -1]],
 a[[[0, 1],
    [1, 2],
    [2, 0]]])

(array([1, 2, 3]),
 array([1, 3]),
 array([3, 2, 1]),
 array([3, 3, 3, 3]),
 array([[1, 2],
        [2, 3],
        [3, 1]]))

In [ ]:
# 2 次元配列の例
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
row_indices = np.array([[2, 0],
                        [0, 1]])
col_indices = np.array([[1, 2],
                        [1, 0]])
(a[[2, 0], [1, 2]],  # 1 次元配列を返す
 a[row_indices, col_indices],  # 2 次元配列を返す
 a[[2, 0]],  # 2 次元配列を返す（a[[[2], [0]], :] の省略形とみなされ、 : は [[0, 1, 2]] と変換され、ブロードキャストされる）
 a[:, [1, 2]],  # 2 次元配列を返す（ : は [[0], [1], [2]] と変換され、ブロードキャストされる）
 a[row_indices, -1],  # 2 次元配列を返す（ブロードキャストされる）
 )

(array([8, 3]),
 array([[8, 3],
        [2, 4]]),
 array([[7, 8, 9],
        [1, 2, 3]]),
 array([[2, 3],
        [5, 6],
        [8, 9]]),
 array([[9, 3],
        [3, 6]]))

2 次元以上の配列に対するインデックス参照で、 2 次元以上の配列が得られるようなファンシーインデックスを構成する作業は煩雑となりやすい。この作業に役立つヘルパー関数が提供されている。

``` python
numpy.ix_(*args)
```

この関数は、N 個のネストしていないシーケンスを受け取り、それぞれ N 次元を持つ N 個の配列のタプルを返す。各配列は、ブロードキャストされるとちょうど格子点を抽出するような形状になっている。

次のコードは、 2 つのリスト `[2, 0], [0, 1]` をこの関数に渡した戻り値を `ixgrid` 変数に格納している:

In [ ]:
# 2個のリストを渡す場合
ixgrid = np.ix_([2, 0],
                [0, 1])
print(ixgrid)  # 2次元配列が2個入ったタプル
print('-' * 30)
# ixgrid に入った2個の配列に対してブロードキャストして形状を揃えた結果をみる
np.broadcast_arrays(*ixgrid)

(array([[2],
       [0]]), array([[0, 1]]))
------------------------------


(array([[2, 2],
        [0, 0]]),
 array([[0, 1],
        [0, 1]]))

この `ixgrid` を使い `a[ixgrid]` でインデックス参照すると、次のような配列が返される:

``` text
[a[2, 0], a[2, 1],
 a[0, 0], a[0, 1]]
```

In [ ]:
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
ixgrid = np.ix_([2, 0],
                [0, 1])
a[ixgrid]

array([[7, 8],
       [1, 2]])

つまり、`a[np.ix_([2, 0], [0, 1])` は、次のような格子点が抽出されたものになる:

``` text

     ⓪  ①  ２
   ┌─┬─┬─┐
⓪ │❶│❷│３│
   ├─┼─┼─┤
１ │４│５│６│
   ├─┼─┼─┤
② │❼│❽│９│
   └─┴─┴─┘
```

型変換
------

### デフォルトのデータ型 ###

`numpy.array()` 関数は、デフォルトでは、第 1 引数に渡された入力データ（配列やリストなど）内のすべての要素を保持できる、最も一般的なデータ型を選択しようとする。

  * 整数のリストを渡すと、 `int`（Numpy の型名、以下同様）になる。 `int` では値が保持されない要素が含まれる場合、 `float64` になる。 `float64` でも値が保持されない要素が含まれる場合は、 `object` になる。
  * 浮動小数点数のリストを渡すと、 `float64` になる。非数 `numpy.nan` と無限大 `numpy.inf` は浮動小数点数の表現として扱われる。
  * 整数と浮動小数点数が混在するリストを渡すと、より広い範囲を表現できる `float64` になる。
  * 複素数のリストを渡すと、`complex128` になる。
  * ブール値のリストを渡すと、`bool` になる。
  * 文字列を含むリストを渡すと、最も長い文字列を格納できる `unicode` になる。
  * 文字列と数値のリストを渡すと、より広い範囲を表現できる `unicode` になる。

In [64]:
print(f'{np.array([1, 2, 3]).dtype = }')
print(f'{np.array([1, 2.0, 3]).dtype = }')
print(f'{np.array(["aa", "あああ"]).dtype = }')  # 最も長い文字列 "あああ" を格納できる長さ 3 の の固定長 Unicode 文字列型になる
print(f'{np.array([1, 2, "3"]).dtype = }')  # 64 ビット符号あり整数を表現できる長さ 21 の固定長 Unicode 文字列型になる

np.array([1, 2, 3]).dtype = dtype('int64')
np.array([1, 2.0, 3]).dtype = dtype('float64')
np.array(["aa", "あああ"]).dtype = dtype('<U3')
np.array([1, 2, "3"]).dtype = dtype('<U21')


### 型変換を行うメソッド ###

``` python
ndarray.astype(dtype, order='K', casting='unsafe', subok=True, copy=True)
```

この `numpy.ndarray` メソッドは、第 1 引数に指定された dtype に型変換（キャスト）した配列のコピーを返す。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `dtype` | 指定された dtype に型変換を行う |
| `order` | 配列のメモリレイアウトを文字列 `'C'`, `'F'`, `'A'`, `'K'` で指定する |
| `casting` | どのような種類の型変換が許可されるかについてのポリシーを文字列で指定する<br /><br />・`'no'`: データ型をまったく変換しない<br /><br />・`'equiv'`: バイトオーダーの変更のみが許可される<br /><br />・`'safe'`: 値を保持できる型変換のみが許可される<br /><br />・`'same_kind'`: `float64` から `float32` へのような同じ種類への変換のみが許可される<br /><br />・`'unsafe'`: あらゆる型変換が行われる |

整数同士、あるいは、浮動小数点数同士でビット数の大きい型への変換は、値が保持される。しかし、ビット数の小さい型への変換は、オーバフローが発生して値が壊れる可能性がある。

In [54]:
a = np.array([100, 1000, 10000], dtype='i2')
assert a.dtype == np.int16  # 16 ビット符号あり整数型

# 32 ビット符号あり整数型への型変換は値が保持される
b = a.astype('i4')
print(f'{b = }')

# 8 ビット符号あり整数型への型変換は値が保持されない
c = a.astype('i1')
print(f'{c = }')

b = array([  100,  1000, 10000], dtype=int32)
c = array([100, -24,  16], dtype=int8)


浮動小数点数型から整数型への変換は、小数点以下切り捨てとなる。

In [34]:
a = np.array([1.2, 2.4, 3.6])
assert a.dtype == np.float64
a.astype(int)

array([1, 2, 3])

なお、普通の配列（`numpy.ndarray` オブジェクト）では、個別にデータ型を指定することはできない。たとえば、 2 次元配列の列ごとにデータ型を指定するようなことはできない。

代入
----

Numpy での代入は、 Python での代入とは異なる独自の操作となる。

【左辺で単一の要素を参照している場合】  
この場合、参照している要素が変更される。右辺は Python のスカラー値でなければならない。左辺の配列の dtype への暗黙的な型変換も行われる。浮動小数点数から整数への変換では、小数以下の切り捨てとなる。型変換に失敗すると、 `ValueError` 例外が発生する。

In [ ]:
a = np.array([[1, 2, 3],
              [4, 5, 6]])
a[0, 1] = 20.8
a

array([[ 1, 20,  3],
       [ 4,  5,  6]])

【左辺で部分配列を参照している場合】  
この場合、右辺も配列で 2 つの配列の形状が一致しているか、あるいは、両辺がブロードキャスト可能である必要がある。位置が対応する要素ごとに代入が行われる。必要により、暗黙的なブロードキャストと暗黙的な型変換が行われる。ブロードキャストや型変換に失敗すると、 `ValueError` 例外が発生する。

左辺で参照されている部分配列と、右辺の配列（または参照されている部分配列）の形状が一致している場合は、そのまま位置が対応する要素ごとに代入が行われる。

In [ ]:
# 左辺で参照している配列と右辺の配列で次元数と形状が同じである場合
a = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11],
              [12, 13, 14, 15]])
b = np.array([[100, 101, 102, 103],
              [104, 105, 106, 107],
              [108, 109, 110, 111],
              [112, 113, 114, 115]])
a[np.ix_([2, 3], [0, 1, 2])] = b[np.ix_([0, 1], [0, 1, 2])]
a

array([[  0,   1,   2,   3],
       [  4,   5,   6,   7],
       [100, 101, 102,  11],
       [104, 105, 106,  15]])

左辺でも右辺でも参照されている部分配列がもとの配列において飛び飛びであってもよい。

In [ ]:
# 左辺でスライスが使われている場合
a = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11],
              [12, 13, 14, 15]])
b = np.array([[100, 101],
              [200, 201]])
assert a[::2, ::2].shape == b.shape
a[::2, ::2] = b
a

array([[100,   1, 101,   3],
       [  4,   5,   6,   7],
       [200,   9, 201,  11],
       [ 12,  13,  14,  15]])

左辺で参照されている部分配列と、右辺の配列（または参照されている部分配列）の形状が一致していない場合、左辺で参照されている部分配列の形状に揃えるようにブロードキャストされる。ブロードキャスト可能でなければ `ValueError` 例外が発生する。

In [ ]:
# 左辺で参照している配列と右辺の配列で次元数と形状が異なる場合
a = np.array([[1, 1, 1],
              [2, 2, 2],
              [3, 3, 3]])
assert a[1:].shape == (2, 3)
a[1:] = np.array([-1, -1, -1])  # ブロードキャストが行われる
a

array([[ 1,  1,  1],
       [-1, -1, -1],
       [-1, -1, -1]])

上記のコードでは、左辺で参照されている部分配列の形状が `(2, 3)` なので、右辺の配列は次のようにブロードキャストされる。

``` text
1. 入力:
    [-1, -1, 1-]      【形状 (3,)】
2. 次元数を 2 に揃える:
    [[-1, -1, -1]]    【形状 (1, 3)】
3. 0 軸での配列のサイズを 2 に揃える:
    [[-1, -1, -1],    【形状 (2, 3)】
     [-1, -1, -1]]
```

スカラーは常にブロードキャスト可能なので、右辺のスカラーは左辺で参照されている部分配列を埋めるように代入される。

In [ ]:
# 右辺がスカラーの場合
a = np.array([[1, 1, 1],
              [2, 2, 2],
              [3, 3, 3]])
a[1:] = -1  # ブロードキャストが行われる
a

array([[ 1,  1,  1],
       [-1, -1, -1],
       [-1, -1, -1]])

代入はアンパックもサポートしている。

In [ ]:
a = np.array([[10, 20],
              [30, 40]])
b, c = a
[x, y], [z, w] = a
b, c, x, y, z, w

(array([10, 20]), array([30, 40]), 10, 20, 30, 40)

以上のことは、整数やスライスを使うインデックスでも、ブーリアンインデックスでも、ファンシーインデックスでも、同様である。配列の代入では、右辺がビューを返すインデクス参照を使っていても常に値のコピーが代入されるし、左辺がコピーを返すインデクス参照を使っていても配列自体が変更される。

In [66]:
a = np.arange(1, 5)
print(f'{a = }')
b = np.arange(100, 105)
print(f'{b = }')
a[[True, False, False, True]] = b[::4]
assert not np.shares_memory(a, b)
print(f'{a = }')

a = array([1, 2, 3, 4])
b = array([100, 101, 102, 103, 104])
a = array([100,   2,   3, 104])


一方、左辺に関してビューが作成されていた場合、代入はメモリを共有する別の配列も変更することに注意する。ブーリアンインデックスやファンシーインデックスなどによってコピーを作成していた場合は、その一方の配列に対する代入によって他方の配列が変更されることはない。

In [ ]:
a = np.arange(5)
b = a[:]  # ビューを返す
c = a[[True, True, True, True, True]]  # コピーを返す
d = a[[0, 1, 2, 3, 4]]  # コピーを返す
assert np.shares_memory(a, b) and not np.shares_memory(a, c) and not np.shares_memory(a, d)
a[[True, True, False, False, True]] = 100
a, b, c, d

(array([100, 100,   2,   3, 100]),
 array([100, 100,   2,   3, 100]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]))

整数やスライスを使うインデックス参照であっても、そのコピーを取れば、代入によって変更されることを防ぐことができる。ただし、メモリ消費量が増えることに注意する。

In [ ]:
a = np.arange(5)
b = a[:].copy()
a[::2] = 100
a, b

(array([100,   1, 100,   3, 100]), array([0, 1, 2, 3, 4]))

配列を操作する関数・メソッド
----------------------------

### 形状変更・リスト化 ###

``` python
numpy.reshape(a, newshape, order='C')
ndarray.reshape(shape, /, *, order='C')
```

`numpy.reshape()` は、入力された配列と同じデータを含む新しい形状の配列のビューを返す NumPy 関数である。

| 引数 | 意味 |
|:---|:---|
| `a` | 操作の対象となる配列または array-like オブジェクト。この関数は、`a` 自体を変更しない |
| `newshape` | 新しい形状。`a` と要素数が一致する形状でないと `ValueError` となる。たとえば、`a` の要素数が 12 なら `(12,)`, `(1, 12)`, `(3, 4)`, `(2, 2, 3)` など形状を指定できる。<br />`newshape` のタプルには 1 つだけ `-1` を指定でき、`-1` とした次元の長さは他の次元の指定値から自動的に決定される。1 次元配列を作る場合は `newshape` に単独の整数を指定<br />できる |
| `order` | 配列のメモリレイアウトと `a` から要素を取る順番を同時に指定する。`'C'` か `'F'` か `'A'` を指定できる。`'A'` を指定した場合、`a` がカラムメジャーオーダーなら維持されるが、そ<br />れ以外なら常にローメジャーオーダーになる。デフォルトは `'C'` |

`numpy.reshape()` 関数の第 1 引数は配列に限られず array-like オブジェクトを受け付けるので、配列の生成にも使用できる（ただし dtype を指定できないので配列の生成には `numpy.array()` を使うのが基本）。

`ndarray.reshape()` は、 `numpy.ndarray` メソッドである。 `a.reshape(shape, order)` は `numpy.reshape(a, shape, order)` と等価。ただし、`ndarray.reshape()` メソッドでは、新しい形状の指定方法として、タプルではなく、その要素を個別の引数として渡すことができる。たとえば、`a.reshape(10, 11)` は `a.reshape((10, 11))` と同等となる。

引数 `order` は、もとの配列から要素を取る順番を決めるので、その指定によって戻り値の配列（ビュー）の中身が変わることに注意する。デフォルトでは、もとの配列からローメジャーオーダーの順番で要素を取得し、新しい形状の配列にローメジャーオーダーの順番で要素を埋めていく形となる。 2 次元配列から 2 次元配列へ形状変更する場合、最初の行の左から要素を取得していき、終わったら次の行の左から要素を取得していく、というように要素を取得したら、新しい形状の配列において最初の行の左から要素を埋めていき、行が埋まったら次の行の左から要素を埋めていく、という操作になる。

In [7]:
# ローメジャーオーダーの場合

# 1次元配列→2次元配列
a = np.arange(1, 13)
# [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]
b = np.reshape(a, (3, -1))  # a.reshape(3, 4) と同じ
print(f'{b!r}')

# 2次元配列→2次元配列
c = np.reshape(b, (-1, 3))  # b.reshape(4, 3) と同じ
print(f'{c!r}')

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])
array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])


`order` 引数に `'F'` を指定すると、もとの配列からカラムメジャーオーダーの順番で要素を取得し、新しい形状の配列にカラムメジャーオーダーの順番で要素を埋めていく形となる。

In [8]:
# カラムメジャーオーダーの場合

# 1次元配列→2次元配列
a = np.arange(1, 13)
# [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]
b = np.reshape(a, (3, -1), order='F')  # a.reshape(3, 4, order='F') と同じ
print(f'{b!r}')

# 2次元配列→2次元配列
c = np.reshape(b, (-1, 3), order='F')  # b.reshape(4, 3, order='F') と同じ
print(f'{c!r}')

array([[ 1,  4,  7, 10],
       [ 2,  5,  8, 11],
       [ 3,  6,  9, 12]])
array([[ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11],
       [ 4,  8, 12]])


新しい形状の指定として単独の整数 `-1` を指定すると、任意の配列を 1 次元化することができる。

In [ ]:
a = np.arange(1, 13).reshape(3, -1)
print(f"{a!r}")
print("ローメジャーオーダー:")
print(f"{np.reshape(a, -1)!r}")
print("カラムメジャーオーダー:")
a.reshape(-1, order='F')

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])
ローメジャーオーダー:
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
カラムメジャーオーダー:


array([ 1,  5,  9,  2,  6, 10,  3,  7, 11,  4,  8, 12])

【NumPy 2.1 での変更点】  
`numpy.reshape()` 関数の `newshape` 引数は `shape` へ引数名が変更され、互換性維持のため `newshape` キーワード専用引数が追加された。同時に `order` 引数もキーワード専用引数に変更された。 `ndarray.reshape()` メソッドの `shape` 引数は従前のままである。

また、 `numpy.reshape()` 関数と `ndarray.reshape()` メソッドに新しい `copy` キーワード専用引数が追加され、`copy=True` を指定すると配列のコピーを返すようになった。 `copy` のデフォルトは `None` で、可能な限り配列のビューを返す。

``` python
numpy.ravel(a, order='C')
ndarray.ravel([order])
ndarray.flatten(order='C')
```

`numpy.ravel(a, order)` は `numpy.reshape(a, -1, order)` と同じ。ただし、 `order` 引数に `'K'` を指定することができる。 `a.ravel(order)` は `numpy.ravel(a, order)` と等価である。

また、 `a.flatten()` は `a.ravel()` と同様に 1 次元化するが、常にコピーを返す。対応する Numpy 関数はない。 `a.flatten()` はメモリを新たに確保するため、`a.ravel()` よりも遅い。

``` python
ndarray.tolist()
```

このメソッドは、配列をネストされたリスト（多重リスト）に変換する。

In [ ]:
a = np.array([[1, 2], [3, 4]])
a.tolist()

[[1, 2], [3, 4]]

### 軸の入れ替え・転置 ###

``` python
numpy.transpose(a, axes=None)
ndarray.transpose(*axes)
ndarray.T
```

配列と同じデータを含む軸を入れ替えた配列（ビュー）を返す。`ndarray.T` は `numpy.ndarray` の属性（読み取り専用プロパティ）であり、 `a.T` は `numpy.transpose(a)` と等価である。

| 引数 | 意味 |
|:---|:---|
| `axes` | 軸の入れ替えを指定するタプルまたはリスト。`None`（デフォルト）の場合は軸の順序が逆になる |

入力が 2 次元配列の場合、数学で言う行列の転置と同じ操作となる。

In [ ]:
a = np.arange(1, 7).reshape(2, -1)
print(f"{a!r}")
np.transpose(a)  # a.transpose(), a.T と同じ

array([[1, 2, 3],
       [4, 5, 6]])


array([[1, 4],
       [2, 5],
       [3, 6]])

上記のコードでは、行方向の軸（0 軸）と列方向の軸（1 軸）を入れ替えて、中の要素が次のように配置される:

``` text
     1軸                                                              0軸
     ─→                                                             ─→
   │a[0, 0] = 1,  a[0, 1] = 2,  a[0, 2] = 3,  np.transpose(a)      │a[0, 0] = 1,  a[1, 0] = 4,
0軸↓a[1, 0] = 4,  a[1, 1] = 5,  a[1, 2] = 6,  ───────→  1軸↓a[0, 1] = 2,  a[1, 1] = 5,
                                                                      a[0, 2] = 3,  a[1, 2] = 6,
```

入力の次元が 3 次元以上の場合、デフォルトでは軸の順序が逆になるが、軸の順番を指定することもできる。

In [ ]:
a = np.arange(1, 9).reshape(2, 2, 2)
print(f"{a!r}")
print(f"{np.transpose(a)!r}")  # a.transpose(), a.T と同じ
np.transpose(a, (0, 2, 1))  # a.transpose(0, 2, 1) と同じ

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])
array([[[1, 5],
        [3, 7]],

       [[2, 6],
        [4, 8]]])


array([[[1, 3],
        [2, 4]],

       [[5, 7],
        [6, 8]]])

上記のコードでまず軸の順番が逆になると、中の要素が次のように配置される:

``` text
          2軸                                                           0軸
          ─→                                                          ─→
   │   │a[0, 0, 0] = 1,  a[0, 0, 1] = 2,                       │   │a[0, 0, 0] = 1,  a[1, 0, 0] = 5,
   │1軸↓a[0, 1, 0] = 3,  a[0, 1, 1] = 4,  np.transpose(a)      │1軸↓a[0, 1, 0] = 3,  a[1, 1, 0] = 7,
0軸│     2軸                               ───────→  2軸│     0軸
   │     ─→                                                   │     ─→
   │   │a[1, 0, 0] = 5,  a[1, 0, 1] = 6,                       │   │a[0, 0, 1] = 2,  a[1, 0, 1] = 6,
   ↓1軸↓a[1, 1, 0] = 7,  a[1, 1, 1] = 8,                       ↓1軸↓a[0, 1, 1] = 4,  a[1, 1, 1] = 8,
```

また、軸の順番を `(0, 2, 1)` に入れ替えると、中の要素が次のように配置される:

``` text
          2軸                                                           1軸
          ─→                                                          ─→
   │   │a[0, 0, 0] = 1,  a[0, 0, 1] = 2,                       │   │a[0, 0, 0] = 1,  a[0, 1, 0] = 3,
   │1軸↓a[0, 1, 0] = 3,  a[0, 1, 1] = 4,  np.transpose(a)      │2軸↓a[0, 0, 1] = 2,  a[0, 1, 1] = 4,
0軸│     2軸                               ───────→  0軸│     1軸
   │     ─→                                                   │     ─→
   │   │a[1, 0, 0] = 5,  a[1, 0, 1] = 6,                       │   │a[1, 0, 0] = 5,  a[1, 1, 0] = 7,
   ↓1軸↓a[1, 1, 0] = 7,  a[1, 1, 1] = 8,                       ↓2軸↓a[1, 0, 1] = 6,  a[1, 1, 1] = 8,
```

入力が 1 次元配列の場合、軸が 1 つしかないので `numpy.transpose()` はもとの配列そのままのビューを返すだけとなる。線形代数では、縦ベクトルと横ベクトルの区別があって転置で互いに変換されるが、 `numpy.ndarray` の 1 次元配列はそのような区別がなく、数学で言うところのベクトルとは厳密には異なる。

In [ ]:
a = np.array([1, 2, 3])
np.transpose(a)  # 1 次元配列ではもとの配列そのままのビューを返す

array([1, 2, 3])

縦ベクトルや横ベクトルを区別して表現するには、それぞれ列数が 1 である 2 次元の配列と、行数が 1 である 2 次元配列を用いる。たとえば、`numpy.arange(5)[numpy.newaxis, :].T` は `numpy.arange(5)[:, numpy.newaxis]` と同等である。

In [ ]:
a = np.arange(5)[np.newaxis, :].T
assert a.shape == np.arange(5)[:, np.newaxis].shape == (5, 1)
a

array([[0],
       [1],
       [2],
       [3],
       [4]])

### 分割 ###

``` python
numpy.split(ary, indices_or_sections, axis=0)
numpy.array_split(ary, indices_or_sections, axis=0)
```

これらの関数は、第 1 引数の配列 `ary` を第 2 引数と第 3 引数で指定する方法で分割する。戻り値は、分割された配列（もとの配列のビュー）のリスト。

  * 第 2 引数が整数 `N` の場合、 `ary` は第 3 引数の軸に沿って `N` 等分に分割される。このような分割が不可能な場合（つまり割り切れないような整数を指定した場合）は、 `numpy.split()` 関数では `ValueError` 例外が発生するのに対して、 `numpy.array_split()` 関数ではできるだけ等分割になるように分割する。
  * 第 2 引数がソートされた整数の 1 次元配列（array-like オブジェクトも可）である場合、そのエントリーは `ary` が軸に沿って分割される位置を示す。例えば、`[2, 3]` と指定した場合、 `axis=0` なら、3 つの配列 `ary[:2]` と `ary[2:3]` と `ary[3:]` に分割される。インデックスが軸に沿った配列の次元を超える場合、それに応じて空のサブ配列が返される。
  * 第 3 引数の軸はデフォルトで `0` 軸。存在しない軸を指定すると `ValueError` 例外が発生する。

In [ ]:
# 行列の行を 2 等分する
a = np.arange(8).reshape(4, 2)
b, c = np.split(a, 2)
a, b, c

(array([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]]),
 array([[0, 1],
        [2, 3]]),
 array([[4, 5],
        [6, 7]]))

In [ ]:
# 3 次元配列を各行列の列で 3 等分する
a = np.full((2, 2, 3), [0, 1, 2])
b, c, d = np.split(a, 3, axis=2)
a, b, c, d

(array([[[0, 1, 2],
         [0, 1, 2]],
 
        [[0, 1, 2],
         [0, 1, 2]]]),
 array([[[0],
         [0]],
 
        [[0],
         [0]]]),
 array([[[1],
         [1]],
 
        [[1],
         [1]]]),
 array([[[2],
         [2]],
 
        [[2],
         [2]]]))

In [5]:
# np.array_split() は割り切れない整数でも、できるだけ等分割になるように分割する
a = np.arange(8)
# [0, 1, 2, 3, 4, 5, 6, 7]
np.array_split(a, 3)

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7])]

In [ ]:
# ベクトルを指定した位置で分割する
a = np.arange(10)
b, c, d = np.split(a, [2, 10])
a, b, c, d  # d は空のベクトルとなる

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 1]),
 array([2, 3, 4, 5, 6, 7, 8, 9]),
 array([], dtype=int64))

In [ ]:
# 行列の列を指定した位置で分割する
a = np.arange(10).reshape(2, -1)
b, c, d = np.split(a, [2, 6], axis=1)
a, b, c, d  # d は空の 2x0 行列となる

(array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]]),
 array([[0, 1],
        [5, 6]]),
 array([[2, 3, 4],
        [7, 8, 9]]),
 array([], shape=(2, 0), dtype=int64))

``` python
numpy.vsplit(ary, indices_or_sections)
```

この関数は `numpy.split(ary, indices_or_sections, 0)` と同じ。

``` python
numpy.hsplit(ary, indices_or_sections)
```

この関数は

  * `ary` が 1 次元配列なら `numpy.split(ary, indices_or_sections, 0)` と同じ
  * `ary` が 2 次元以上の配列なら `numpy.split(ary, indices_or_sections, 1)` と同じ

``` python
numpy.dsplit(ary, indices_or_sections)
```

この関数は `numpy.split(ary, indices_or_sections, 2)` と同じ。 `ary` が 1 次元配列や 2 次元配列の場合は `ValueError` 例外が発生する。

### 結合 ###

``` python
numpy.concatenate((a1, a2, ...), axis=0, out=None, dtype=None, casting="same_kind")
```

この関数は、配列または array-like オブジェクトのシーケンスに対して、既存の軸 `axis` 沿って結合された新しい配列を返す。 `axis` に存在しない軸を指定すると `ValueError` 例外が発生する。 `axis=-1` は最後の次元（軸）を指定する。 `axis` 以外の軸ではサイズが一致していないと `ValueError` 例外が発生する。

`out` 引数に出力先の配列を指定できる。指定する場合は、結合した結果の配列と形状が一致する必要がある。

`casting` 引数には、結合に際して型変換が必要な場合に型変換のポリシーを指定する。 `casting` のデフォルト値は `same_kind`。

In [ ]:
# 2 つの異なる型のベクトルの結合
a = np.arange(0, 5)
b = np.arange(-1., -6, -1.)
np.concatenate((a, b))  # 型変換が行われる

array([ 0.,  1.,  2.,  3.,  4., -1., -2., -3., -4., -5.])

In [ ]:
# 2 つの行列の結合
a = np.array([[1, 2],
              [3, 4]])
b = np.array([[5, 6]])
np.concatenate((a, b)), np.concatenate((a, b.T), axis=-1)

(array([[1, 2],
        [3, 4],
        [5, 6]]),
 array([[1, 2, 5],
        [3, 4, 6]]))

``` python
numpy.vstack(tup, *, dtype=None, casting='same_kind')
```

この関数は

  * `tup` が形状 `(N,)` の 1 次元配列のシーケンスなら、その配列がそれぞれ `(1, N)` に形状変更された上で最初の軸に沿った `numpy.concatenate()` を適用されるのと同じ
  * `tup` が 2 次元以上の配列のシーケンスなら、 `numpy.concatenate(tup)` と同じ

`numpy.vsplit()` 関数で分割された配列を再構築することができる。

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
np.vstack((a, b))  # np.concatenate((a[np.newaxis], b[np.newaxis])) と同じ

array([[1, 2, 3],
       [4, 5, 6]])

``` python
numpy.hstack(tup, *, dtype=None, casting='same_kind')
```

この関数は `numpy.concatenate(tup, axis=1)` と同じ。 `numpy.hsplit()` 関数で分割された配列を再構築することができる。

``` python
numpy.dstack(tup)
```

この関数は

  * `tup` が形状 `(N,)` の 1 次元配列のシーケンスなら、その配列がそれぞれ `(1, N, 1)` に形状変更された上で 2 軸（3番目の軸）に沿った `numpy.concatenate()` を適用されるのと同じ
  * `tup` が形状 `(M, N)` の 2 次元配列のシーケンスなら、その配列がそれぞれ `(M, N, 1)`  に形状変更された上で 2 軸（3番目の軸）に沿った `numpy.concatenate()` を適用されるのと同じ
  * `tup` が 3 次元以上の配列なら `numpy.concatenate(tup, axis=2)` と同じ

`numpy.dsplit()` 関数で分割された配列を再構築することができる。

In [ ]:
# 1 次元配列のシーケンスの場合
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
np.dstack((a, b))  # np.concatenate((a.reshape(1, -1, 1), b.reshape(1, -1, 1)), axis=2) と同じ

array([[[1, 4],
        [2, 5],
        [3, 6]]])

In [ ]:
# 2 次元配列のシーケンスの場合
a = np.array([[ 1,  2,  3],
              [ 4,  5,  6]])
b = np.array([[11, 12, 13],
              [14, 15, 16]])
np.dstack((a, b))  # np.concatenate((a[:, :, np.newaxis], b[:, :, np.newaxis]), axis=2) と同じ

array([[[ 1, 11],
        [ 2, 12],
        [ 3, 13]],

       [[ 4, 14],
        [ 5, 15],
        [ 6, 16]]])

``` python
numpy.block(arrays)
```

この関数は、配列または array-like オブジェクトの（ネストされた）リストから新しい配列を組み立てる。

最も内側のリストのブロックは最後の次元（`-1`）に沿って `numpy.concatenate()` が適用され、次に最後から 2 番目の次元（`-2`）に沿って `numpy.concatenate()` が適用され、最も外側のリストに到達するまで続く。

1 次元または 2 次元の配列のリストでは、リストの内側では横方向に配列を配置することになり、ネストされたリストの外側では縦方向に配列を配置することになる。

In [ ]:
# 1 次元配列の場合
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
(np.block([a, b]),
 np.block([[a],
           [b]]))

(array([1, 2, 3, 4, 5, 6]),
 array([[1, 2, 3],
        [4, 5, 6]]))

In [ ]:
# 2 次元配列の場合
a = np.arange(1, 7).reshape(2, 3)
b = np.arange(11, 17).reshape(2, 3)
(np.block([a, b]),
 np.block([[a],
           [b]]))

(array([[ 1,  2,  3, 11, 12, 13],
        [ 4,  5,  6, 14, 15, 16]]),
 array([[ 1,  2,  3],
        [ 4,  5,  6],
        [11, 12, 13],
        [14, 15, 16]]))

リストの中のブロックは、通常のルールを使用してブロードキャストされることはない。ただし、先頭に軸が追加され、その次元ではサイズが 1 の配列とするような形状変更によりブロックの次元数を一致させるということは行われる。これは主にスカラーを扱う場合に便利で、`np.block([v, 1])`（`v.ndim == 1`）のようなコードが有効であることを意味する。

In [ ]:
a = np.block([1, 2, 3])  # np.hstack([[1], [2], [3]]) と同じ
a, np.block([a, 4, 5])

(array([1, 2, 3]), array([1, 2, 3, 4, 5]))

``` python
numpy.stack(arrays, axis=0, out=None, *, dtype=None, casting='same_kind')
```

この関数は、配列または array-like オブジェクトのシーケンスに対して、新しい軸 `axis` に沿って積み重ねたような新しい配列を返す。戻り値の配列は、入力配列よりも次元数が 1 つ多くなる。

入力配列は形状が一致している必要がある。形状が一致していない場合は、 `ValueError` 例外が発生する。

`axis` 引数は、出力される結合配列における軸のインデックスを指定する。例えば、`axis=0`（デフォルト）の場合は最初の次元になり、`axis=-1` の場合は最後の次元になる。

その他の引数は `numpy.concatenate()` と同様。

1 次元配列のシーケンス `arrays` にする `numpy.stack(arrays)` は `numpy.vstack(arrays)` と等価である。

2 次元の配列のシーケンス `arrays` にする `numpy.stack(arrays, axis=2)` は `numpy.dstack(arrays)` と等価である。

In [ ]:
# 1 次元配列同士のスタック
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
np.stack((a, b)), np.stack((a, b), axis=-1)

(array([[1, 2, 3],
        [4, 5, 6]]),
 array([[1, 4],
        [2, 5],
        [3, 6]]))

In [ ]:
# 2 次元配列同士のスタック
a = np.array([[ 1,  2,  3],
              [ 4,  5,  6]])
b = np.array([[11, 12, 13],
              [14, 15, 16]])
np.stack((a, b)), np.stack((a, b), axis=-1)

(array([[[ 1,  2,  3],
         [ 4,  5,  6]],
 
        [[11, 12, 13],
         [14, 15, 16]]]),
 array([[[ 1, 11],
         [ 2, 12],
         [ 3, 13]],
 
        [[ 4, 14],
         [ 5, 15],
         [ 6, 16]]]))

### 削除 ###

``` python
numpy.delete(arr, obj, axis=None)
```

この関数は、入力された配列に対して軸に沿った部分配列が削除された新しい配列（コピー）を返す。

| 引数 | 意味 |
|:---|:---|
| `arr` | 操作の対象となる配列または array-like オブジェクト。この関数は、`arr` 自体を変更しない |
| `obj` | 削除すべき部分配列を指定するインデックスを、整数やスライス、リスト（配列）で指定する |
| `axis` | `obj` の指定がどの軸での意味であるかを指定する。`axis` が `None`（デフォルト）の場合、`obj` は 1 次元化された配列に適用される |

In [ ]:
# 行の削除
a = np.arange(12).reshape(3, 4)
b = np.delete(a, 1, axis=0) # 2 行目を削除
c = np.delete(a, [0, 2], axis=0) # 1 行目と 3 行目を削除
d = np.delete(a, slice(2), axis=0) # 1 行目と 2 行目を削除
a, b, c, d  # a は変更されない

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([[ 0,  1,  2,  3],
        [ 8,  9, 10, 11]]),
 array([[4, 5, 6, 7]]),
 array([[ 8,  9, 10, 11]]))

`numpy.s_[]` を使うとスライスに使用する添字表記 `[start:stop:step]` を記述できる。

In [ ]:
# 列の削除
a = np.arange(12).reshape(3, 4)
b = np.delete(a, 1, axis=1) # 2 列目を削除
c = np.delete(a, [1, 2], axis=1) # 2 列目と 3 列目を削除
d = np.delete(a, np.s_[::2], axis=1) # 1 行目と 3 行目を削除
a, b, c, d  # a は変更されない

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([[ 0,  2,  3],
        [ 4,  6,  7],
        [ 8, 10, 11]]),
 array([[ 0,  3],
        [ 4,  7],
        [ 8, 11]]),
 array([[ 1,  3],
        [ 5,  7],
        [ 9, 11]]))

`axis` が `None` の場合、配列を 1 次元に平坦化したものに対して削除が行われる。

In [4]:
# axis=None の場合
a = np.arange(12).reshape(3, 4)
# [[ 0  1  2  3]
#  [ 4  5  6  7]
#  [ 8  9 10 11]]
np.delete(a, slice(9, None))  # 1 次元化された配列の 10 番目以降の要素が削除される

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

### 挿入・追加 ###

``` python
numpy.insert(arr, obj, values, axis=None)
```

この関数は、指定された軸の上の指定されたインデックスの前に値が挿入された新しい配列（コピー）を返す。

| 引数 | 意味 |
|:---|:---|
| `arr` | 操作の対象となる配列または array-like オブジェクト。この関数は、 `arr` 自体を変更しない |
| `obj` | 挿入位置となるインデックスを、整数やスライス、シーケンスで指定する |
| `values` | `arr` に挿入する値。単一の値または array-like オブジェクトを指定できる |
| `axis` | `obj` の指定がどの軸での意味であるかを指定する。`axis` が `None`（デフォルト）の場合、`obj` は 1 次元化された配列に適用される |

`axis` が整数の場合、挿入は以下のように行われる。

  1. もとの配列の `axis` 上のサイズを `obj` の個数分増やしただけの形状で戻り値となる配列を用意する。
  2. `values` の `axis` 上のサイズが `obj` の個数と一致し、かつ、`values` の他のサイズが対応する `arr` の各次元のサイズと一致する場合、 `values` の `axis` 上の部分配列と `obj` がペア対応して `axis` 上の `obj` の各位置にそれぞれ部分配列を挿入する。
  3. 2 に該当しない場合
      * `obj` が整数（または 1 個の整数だけを持つスライス、シーケンス）なら、 `axis` 上の `obj` の位置に `values` 全体を挿入する。もし `values` の `axis` 以外のサイズと対応する `arr` の各次元のサイズが一致しないなら、 `values` に対してブロードキャストを行ったうえで `values` 全体を挿入する。ブロードキャスト不可能なら `ValueError` 例外が発生する。
      * `obj` がスライスやシーケンスなら、 `values` に対してブロードキャストを行ったうえで 2 の処理をする。 `values` がブロードキャス不可能なら、 `ValueError` 例外が発生する。
  4. `values` の型が `arr` の型と異なる場合、 `values` は `arr` の型に暗黙的に変換される。

この関数は、 `arr` 自体を変更しない。

``` text
【行列の列に挿入する場合】

  入力配列             新しい配列
┏━━━━┓          ┏━━━━┓
┃ [0, 1] ┃          ┃ [0, 1] ┃
┣━━━━┫ ⇒       ┠────┨    ┏━━━━┓      values
┃ [2, 3] ┃    obj[0]┃        ┃ ← ┃ [4, 5] ┃   ┏━━━━┓
┗━━━━┛          ┣━━━━┫    ┗━━━━┛ ↖ ┃ [4, 5] ┃
                      ┃ [2, 3] ┃                   ┣━━━━┫
                      ┠────┨    ┏━━━━┓   ┃ [6, 7] ┃
                obj[1]┃        ┃ ← ┃ [6, 7] ┃ ↙ ┗━━━━┛
                      ┗━━━━┛    ┗━━━━┛
                          ⇓
                      ┏━━━━┓
                      ┃ [0, 1] ┃
                      ┣━━━━┫
                      ┃ [4, 5] ┃
                      ┣━━━━┫
                      ┃ [2, 3] ┃
                      ┣━━━━┫
                      ┃ [6, 7] ┃
                      ┗━━━━┛
```

In [33]:
# 行の挿入
a = np.array([[0, 1, 2],
              [3, 4, 5]])
b = np.insert(a, [1, 2], [[6, 7, 8], [9, 10, 11]], axis=0)
c = np.insert(a, 0, [[6, 7, 8], [9, 10, 11]], axis=0)  # values 全体を挿入する
d = np.insert(a, 1, 6, axis=0)  # 値を繰り返す形で values を引き延ばす
e = np.insert(a, [1, 2], [6, 7, 8], axis=0)  # values にブロードキャストを行う
b, c, d, e

(array([[ 0,  1,  2],
        [ 6,  7,  8],
        [ 3,  4,  5],
        [ 9, 10, 11]]),
 array([[ 6,  7,  8],
        [ 9, 10, 11],
        [ 0,  1,  2],
        [ 3,  4,  5]]),
 array([[0, 1, 2],
        [6, 6, 6],
        [3, 4, 5]]),
 array([[0, 1, 2],
        [6, 7, 8],
        [3, 4, 5],
        [6, 7, 8]]))

In [41]:
# 列の挿入
a = np.array([[0, 1, 2],
              [3, 4, 5]])
b = np.insert(a, [1, 2], [[6, 7], [8, 9]], axis=1)
c = np.insert(a, 0, [[6, 7], [8, 9]], axis=1)  # values 全体を挿入する
d = np.insert(a, 1, 6, axis=1)  # 値を繰り返す形で values を引き延ばす
e = np.insert(a, [1, 2], [6, 7], axis=1)  # values にブロードキャストを行う
b, c, d, e

(array([[0, 6, 1, 7, 2],
        [3, 8, 4, 9, 5]]),
 array([[6, 8, 0, 1, 2],
        [7, 9, 3, 4, 5]]),
 array([[0, 6, 1, 2],
        [3, 6, 4, 5]]),
 array([[0, 6, 1, 7, 2],
        [3, 6, 4, 7, 5]]))

`axis` が `None` の場合（これがデフォルト）、配列を 1 次元に平坦化したものに対して挿入が行われる。

In [45]:
# axis=None の場合
a = np.array([[0, 1, 2],
              [3, 4, 5]])
np.insert(a, np.s_[::3], [100, 200])

array([100,   0,   1,   2, 200,   3,   4,   5])

``` python
numpy.append(arr, values, axis=None)
```

この関数は、配列の末尾に値を追加された新しい配列（コピー）を返す。

| 引数 | 意味 |
|:---|:---|
| `arr` | 操作の対象となる配列または array-like オブジェクト |
| `values` | 追加する配列または array-like オブジェクト |
| `axis` | 値を追加する軸を指定する。`axis` が `None`（デフォルト）の場合、`obj` は 1 次元化された配列に適用される |

`values` は、 `arr` と次元数が一致する必要があり、また、 `axis` を除いて `arr` と同じ形状である必要がある。 `numpy.insert()` の `values` とは異なり、ブロードキャストされることはない。

`values` の型が `arr` の型と異なる場合、 `values` は `arr` の型に暗黙的に変換される。

この関数は、 `arr` 自体を変更しない。

In [ ]:
a = np.arange(10).reshape(2, -1)
b = np.append(a, a, axis=0)
c = np.append(a, a, axis=1)
a, b, c

(array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]]),
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]]),
 array([[0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9, 5, 6, 7, 8, 9]]))

`axis` が `None` の場合、配列を 1 次元に平坦化したものに対して追加が行われる。

In [ ]:
a = np.arange(10).reshape(2, -1)
np.append(a, a)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### 非数・無限大の変換 ###

``` python
numpy.nan_to_num(x, copy=True, nan=0.0, posinf=None, neginf=None)
```

この関数は、配列（または array-like オブジェクト）に含まれる要素 `numpy.nan` と `numpy.inf` を他の値に置き換えた配列を返す。

| 引数 | 意味 |
|:---|:---|
| `x` | 操作の対象となる配列または array-like オブジェクト |
| `copy` | `True`（デフォルト）の場合、この関数は `x` 自体を変更せず新しい配列を返す。`False` の場合、この関数は `x` 自体を変更し、そのビューを返す |
| `nan` | `numpy.nan` を置き換える値。デフォルトでは `0.0` に置き換える |
| `posinf` | `numpy.inf` を置き換える値。デフォルトでは `x.dtype` で表現できる最大の浮動小数点数値に置き換える |
| `neginf` | `-numpy.inf` を置き換える値。デフォルトでは `x.dtype` で表現できる最小の浮動小数点数値に置き換える |

In [ ]:
a = np.array([1.0, np.nan, np.inf, -np.inf])
b = np.nan_to_num(a)
assert not np.shares_memory(a, b)
a, b

(array([  1.,  nan,  inf, -inf]),
 array([ 1.00000000e+000,  0.00000000e+000,  1.79769313e+308,
        -1.79769313e+308]))

In [ ]:
a = np.array([1.0, np.nan, np.inf, -np.inf])
b = np.nan_to_num(a, copy=False)
assert np.shares_memory(a, b)
a

array([ 1.00000000e+000,  0.00000000e+000,  1.79769313e+308,
       -1.79769313e+308])

数学関数・配列演算
------------------

### ベクトル化 ###

N 次元配列を全ての要素ごとに操作する関数を実装しようとすると、 N 重のループを計算しなければならない。

例えば、全ての要素ごとにステップ関数を計算することを考える。ステップ関数は、次の式で定義される関数である。

$$
\mathrm{step}(x) = \begin{cases}
1 & (x \geqq 0) \\
0 & (x < 0)
\end{cases}
$$

次のコードは、 Python でステップ関数を定義し、2 重のループを使って与えられた 2 次元配列の全ての要素に対しステップ関数を計算し、計算結果を保持する配列を作成している。

In [ ]:
def step(x):
    return 0.0 if x < 0.0 else 1.0

rng = np.random.default_rng()
a = rng.uniform(-3, 3, 6).reshape(2, 3)
b = np.empty((a.shape[0], a.shape[1]))
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        b[i, j] = step(a[i, j])
a, b

(array([[ 0.87754079, -1.16785852,  2.05553315],
        [-2.84727332, -0.35727218,  1.86442993]]),
 array([[1., 0., 1.],
        [0., 0., 1.]]))

Python で多重ループは非常に重い処理となる。そこで、 Numpy は配列処理を C に引き渡して Python ループを排除することで高速化する。この仕組みを**ベクトル化**（vectorization）という。 Python 関数をベクトル化するには、次の関数デコレーターを使う。

``` python
numpy.vectorize(pyfunc=np._NoValue, otypes=None, doc=None, excluded=None, cache=False, signature=None)
```

この関数は、 Python 関数 `pyfunc` をベクトル化した関数を作成する。ベクトル化した関数は、入力に配列を直接渡すことができ、要素ごとに `pyfunc()` を計算した値を要素とする配列を出力する。

上記のコードで `step()` 関数をベクトル化すると、次のようになる:

In [ ]:
@np.vectorize
def step(x):
    return 0.0 if x < 0.0 else 1.0

rng = np.random.default_rng()
a = rng.uniform(-3, 3, 6).reshape(2, 3)
b = step(a)
a, b

(array([[ 2.23372984, -1.51987922, -0.99848158],
        [-2.88167067, -1.05204877,  0.73387746]]),
 array([[1., 0., 0.],
        [0., 0., 1.]]))

このように Python ループを排除できた。ベクトル化の効果を検証すると次のようになる:

In [ ]:
def step(x):
    return 0.0 if x < 0.0 else 1.0

def run_step(a):
    b = np.empty((a.shape[0], a.shape[1]))
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            b[i, j] = step(a[i, j])
    return b

vstep = np.vectorize(step)
rng = np.random.default_rng()
a = rng.uniform(-3, 3, 1000000).reshape(1000, 1000)
%timeit run_step(a)
%timeit vstep(a)

1.47 s ± 341 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
452 ms ± 90.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ユニバーサル関数 ###

`numpy.ufunc` クラスは、ベクトル化された関数の特別な型である。 `numpy.ufunc` のインスタンスである関数オブジェクトを**ユニバーサル関数**（universal function）と呼ぶ。 Numpy の数学関数は、多くがユニバーサル関数である。以降では、ユニバーサル関数ではないベクトル化された関数を「非ユニバーサル関数」と呼ぶ。

ユニバーサル関数は、以下のような性質を持つ。

  * 配列は第 1 引数以降の位置専用引数として入力される。入力配列の個数は関数によって決まっており、 1 個のものは単項関数、 2 個のものは二項関数と呼ぶ。位置専用引数は、配列だけでなく array-like オブジェクトも受け付ける。入力がスカラー値（数値）の場合は、スカラー値が返される。そうでない場合は、新しい配列が返される。ただし、関数によっては複数の結果がタプルで返されることがある。
  * 入力が 2 つの配列である関数（二項関数）では、入力配列の形状が一致していることが必要である。ただし、**ブロードキャスト可能な場合は、暗黙的にブロードキャストが行われる**。つまり、 2 つの入力配列に対して暗黙的に `numpy.broadcast_arrays()` を実行し、返された配列を対象に実際の計算を行う。 2 つの入力配列の形状が一致せず、かつ、ブロードキャスト可能でない場合は、 `ValueError` 例外が発生する。
  * 必要により暗黙的な型変換が行われる。

ユニバーサル関数は、オプションとして共通する引数を受け付ける。共通引数は、 `out` を除いて全てキーワード専用引数である。主なものは次のとおり。

| 引数 | 意味 | デフォルト値 |
|:---|:---|:---|
| `out` | `out` 引数に配列を指定した場合、計算結果がその配列に保存される。`out` の配列の形状は、出力配列の形状に一致させる必要がある。ただし、出力がスカラー値なら、<br />`out` は要素数が 1 である 1 次元配列とする必要がある。また、複数の結果がタプルで返される場合では結果を保存するための配列のタプルとしなければならない | `None` |
| `where` | `where` 引数に配列を指定した場合、その真と評価される要素の位置では出力配列に計算結果が入り、偽と評価される要素の位置では出力配列に `out` の値（`out` が<br /> `None` なら初期値）が入る。全要素が `True` の配列を指定する代わりに単に `True` を指定できる。全要素が `False` の配列を指定する代わりに単に `False` を指定できる | `True` |
| `casting` | どのような種類の型変換が許可されるかについてのポリシーを文字列で指定する。ユニバーサル関数では、デフォルトは `'same_kind'` 指定となることに注意 | `'same_kind'` |
| `order` | 入力から要素を取る順番と出力配列のメモリレイアウトを指定する | `'K'` |
| `dtype` | 出力配列の dtype を明示的に指定する | `None` |

``` python
numpy.frompyfunc(func, /, nin, nout, *[, identity])
```

この関数は、任意の Python 関数を受け取り、ユニバーサル関数を返す。`nin` 引数と `nout` 引数にそれぞれ入力の個数と出力の個数を指定できる。

複数の入力を取る Python 関数をベクトル化するには、`numpy.frompyfunc()` を使う。

ただし、 `numpy.frompyfunc()` 関数が作成するユニバーサル関数は、常に dtype が `object` である配列を出力する。 **`dtype` 引数の指定は無視される**ことに注意する。このため、出力配列に対して `astype()` メソッドで型変換する必要がある。

In [ ]:
def sumsub(x, y):
    return x + y, x - y

usumsub = np.frompyfunc(sumsub, 2, 2)
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
usumsub(x, y, casting='unsafe', dtype=int)  # 型変換を強制する引数を指定しても効果がない

(array([4, 4, 4], dtype=object), array([-2, 0, 2], dtype=object))

### reduce ###

入力が 2 つの配列であるユニバーサル関数（二項関数）は、オブジェクトとして次のメソッドを持つ。

``` python
ufunc.reduce(array, axis=0, **kwargs)
```

このメソッドは、 1 次元以上の配列（または array-like オブジェクト）に対して、 `axis` で指定した軸を削減（reduce）することで配列を分解し、最初の 2 つを取り `ufunc` の入力として渡し、さらにその結果と残りの最初のものを `ufunc` の入力として渡す、という計算を行う。デフォルトでは 0 軸を削減する。 `**kwargs` は `ufunc()` の `**kwargs` に渡される。

`array` が 1 次元配列の場合、 0 軸を削減することで要素に分解し、個々の要素がユニバーサル関数の入力となる。例えば、 `ufunc.reduce([1, 2, 3])` は `ufunc(ufunc(1, 2), 3)` と同じである。

`array` が 2 次元配列の場合、 `axis=0` として 0 軸を削減すると `array` は行に分解され、各行がユニバーサル関数の入力となる。例えば、 `ufunc.reduce([[1, 2, 3], [4, 5, 6]])` は `ufunc([1, 2, 3], [4, 5, 6])` と同じである。

``` text
[[1, 2, 3],   (0軸を削減)→  [1, 2, 3],   [4, 5, 6]
 [4, 5, 6]]
```

`array` が 2 次元配列の場合、 `axis=1` として 1 軸を削減すると `array` は列に分解され、各列がユニバーサル関数の入力となる。例えば、 `ufunc.reduce([[1, 2, 3], [4, 5, 6]], axis=1)` は `ufunc(ufunc([1, 4], [2, 5]), [3, 6])` と同じである。

``` text
[[1, 2, 3],   (1軸を削減)→  [1, 4],   [2, 5],   [3, 6]
 [4, 5, 6]]
```

`array` が 3 次元配列の場合、 `axis=0` として 0 軸を削減すると `array` は内部の行列に分解され、各行列がユニバーサル関数の入力となる。例えば、 `ufunc.reduce([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])` は `ufunc([[1, 2], [3, 4]], [[5, 6], [7, 8]])` と同じである。

``` text
[[[1, 2],   (0軸を削減)→  [[1, 2],    [[5, 6],
  [3, 4]],                  [3, 4]],    [7, 8]]

 [[5, 6],
  [7, 8]]]
```

`array` が 3 次元配列の場合、 `axis=1` として 1 軸を削減すると `array` は内部の行列の行ごとに新しい行列を構成する形で分解され、各行列がユニバーサル関数の入力となる。例えば、 `ufunc.reduce([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], axis=1)` は `ufunc([[1, 2], [5, 6]], [[3, 4], [7, 8]])` と同じである。

``` text
[[[1, 2],   (1軸を削減)→  [[1, 2],    [[3, 4],
  [3, 4]],                  [5, 6]],    [7, 8]]

 [[5, 6],
  [7, 8]]]
```

`array` が 3 次元配列の場合、 `axis=2` として 2 軸を削減すると `array` は内部の行列の列ごとに新しい行列を構成する形で分解され、各行列がユニバーサル関数の入力となる。例えば、 `ufunc.reduce([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], axis=2)` は `ufunc([[1, 3], [5, 7]], [[2, 4], [6, 8]])` と同じである。

``` text
[[[1, 2],   (2軸を削減)→  [[1, 3],    [[2, 4],
  [3, 4]],                  [5, 7]],    [6, 8]]

 [[5, 6],
  [7, 8]]]
```

### 算術演算 ###

Numpy は、配列同士の算術演算をサポートしている。

| 演算 | 関数 | 演算子 | 機能 | 戻り値 |
|:---|:---|:--:|:---|:---|
| 加算 | `add` | `+` | 同じ形状の配列同士で要素ごとに加算する | 配列 |
| 減算 | `subtract` | `-` | 同じ形状の配列同士で要素ごとに減算する | 配列 |
| 乗算 | `multiply` | `*` | 同じ形状の配列同士で要素ごとに乗算する | 配列 |
| 除算 | `divide` | `/` | 同じ形状の配列同士で要素ごとに除算する | 配列 |
| 除算 | `true_divide` | `/` | `divide` の別名 | 配列 |
| 冪 | `power` | `**` | 同じ形状の配列同士で要素ごとに冪乗を計算する | 配列 |
| 商 | `floor_divide` | `//` | 同じ形状の配列同士で要素ごとに商を計算する | 配列 |
| 剰余 | `mod` | `%` | 同じ形状の配列同士で要素ごとに剰余を計算する | 配列 |
| 商と剰余 | `divmod` | | 同じ形状の配列同士で要素ごとに商と剰余を計算する | 配列のタプル |

上記の関数は全てユニバーサル関数である。なお、行列間で要素ごとの乗算、除算はそれぞれアダマール積、アダマール除算と呼ばれる。

例えば、 2 つの配列 `a` と `b` に対して、 `numpy.add(a, b)` と `a + b` は等価である。また、 3 つの配列 `a` と `b` と `c` に対して、 `numpy.add.reduce([a, b, c])` は `numpy.add(numpy.add(a, b), c)` と同じで `a + b + c` と等価である。

In [ ]:
assert np.add.reduce([1, 2, 3]) == np.add(np.add(1, 2), 3) == 6

2 つの入力配列の形状が一致しない場合、ブロードキャスト可能なら、演算の前に暗黙的にブロードキャストが行われる。

例えば、次のコードでは、形状が `(4, 3)` である 2 次元配列と形状が `(3,)` である 1 次元配列との間の加算を行っているが、暗黙的にブロードキャストが先に行われ、左側の配列の形状が `(4, 3)` に揃えられてから、要素ごとの計算が行われる（画像は公式ドキュメントより引用）。

In [ ]:
a = np.broadcast_to([[0], [10], [20], [30]], (4, 3))  # 2次元配列（明示的なブロードキャストを使って配列を生成している）
b = np.array([1, 2, 3])  # 1次元配列
a + b  # 加算において暗黙的にブロードキャストが行われる

array([[ 1,  2,  3],
       [11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

![](https://numpy.org/doc/stable/_images/broadcasting_2.png)

このブロードキャストでは、まず次元数が左側の配列から採用されて右側の配列の形状が `(1, 3)` となり、次に 0 軸での配列のサイズも左側の配列から採用されて右側の配列の形状が `(4, 3)` となる。この時点で配列の形状が揃ったのでブロードキャストの手順が完了する。

もし右側の配列の形状が `(4,)` であった場合は、ブロードキャストが失敗し、 `ValueError` 例外が発生する。この場合、上記の手順で右側の配列の形状が `(4, 4)` と変形できるが、今度は 1 軸での配列のサイズを `4` に揃える必要があり、左側の配列では 1 軸での配列のサイズが `3` で `1` ではないので 1 軸方向に引き延ばすことができないのである（単純にもう 1 列追加することはブロードキャストの規則にない）。

![](https://numpy.org/doc/stable/_images/broadcasting_3.png)

暗黙的なブロードキャストは、スカラーに対しても有効である。つまり、スカラーと配列の間で上記の算術演算を実行すると、スカラーは形状が他方の配列の形状に揃えた配列に変換され、その上で実際の演算が行われる。配列とスカラーの間では、常にブロードキャスト可能である。

例えば、配列とスカラーとの間の乗算（スカラー乗算）は以下のようになる。

In [ ]:
np.array([1, 2, 3]) * 2  # np.array([1, 2, 3]) * np.array([2, 2, 2]) と同じ

array([2, 4, 6])

![](https://numpy.org/doc/stable/_images/broadcasting_1.png)

暗黙的ブロードキャストは、形状の異なる配列同士の演算を、明示的に変換を指示しなくても簡潔な形で書けることが利点ではあるが、コードが分かりにくくなることがある。出力配列の形状に十分注意する必要がある。

例えば、形状が `(4, 1)` である 2 次元配列と形状が `(3,)` である 1 次元配列の間の加算では、暗黙的なブロードキャストが行われるため、形状が `(4, 3)` の 2 次元配列を返す。

In [ ]:
a = np.array([0, 10, 20, 30])[:, np.newaxis]
b = np.array([1, 2, 3])
a + b

array([[ 1,  2,  3],
       [11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

![](https://numpy.org/doc/stable/_images/broadcasting_4.png)

演算によって暗黙的な型変換が行われる場合がある。

  * dtype が整数の配列同士の `/` 演算子による除算は、 `float64` 型に変換される。
  * dtype が整数と浮動小数点数の配列同士を被演算子とする `+`、`-`、`*`、`**`、`//` 演算は、 `float64` 型に変換される。
  * 整数同士や浮動小数点数同士でも、ビット数が異なる場合はビット数の大きい方の型に合わせて変換される。

In [ ]:
a_int = np.array([10, 20, 30])
a_float = np.array([1.0, 2.0, 3.0], dtype=np.float64)
b_float32 = np.array([1.5, 2.5, 3.5], dtype=np.float32)
assert (a_int / a_int).dtype == np.float64      # [1., 1., 1.]
assert (a_int + a_float).dtype == np.float64    # [11., 22., 33.]
assert (a_int - a_float).dtype == np.float64    # [ 9., 18., 27.]
assert (a_int * a_float).dtype == np.float64    # [10., 40., 90.]
assert (a_int ** a_float).dtype == np.float64   # [1.0e+01, 4.0e+02, 2.7e+04]
assert (a_int // a_float).dtype == np.float64   # [10., 10., 10.]
assert (a_float + b_float32).dtype == np.float64  # [2.5, 4.5, 6.5]

累算代入演算子（または複合代入演算子）もサポートされている。

In [ ]:
a = np.arange(0, 50, 10)
# [ 0, 10, 20, 30, 40]
a[[1, 3]] += 1
a

array([ 0, 11, 20, 31, 40])

### ゼロ除算 ###

Python では、ゼロで割ると `ZeroDivisionError` 例外が発生してプログラムの実行が止まる。しかし、コンピューターによる数値計算で `a / b` のような除算を行うときに、たまにしか起こらない例外のためにいちいち `b` がゼロかどうか確認するのは面倒であるし、速度的にも不利となる。

そこで、 Numpy では、以下の規則により `ZeroDivisionError` 例外が発生しないようにしている。

  * 非ゼロの値を `0.0` で割った結果は `numpy.inf` となる。
  * 非ゼロの値を `-0.0` で割った結果は `-numpy.inf` となる。
  * ゼロを `0.0` または `-0.0` で割った結果は `numpy.nan` となる。

こうしてゼロ除算で止まらないが、 `RuntimeWarning`（実行時の警告）は出る。

In [ ]:
np.divide(1.0, 0.0), np.divide(1.0, -0.0), np.divide(0.0, 0.0)

<ipython-input-131-722c11b40630>:1: RuntimeWarning: divide by zero encountered in divide
  np.divide(1.0, 0.0), np.divide(1.0, -0.0), np.divide(0.0, 0.0)
<ipython-input-131-722c11b40630>:1: RuntimeWarning: invalid value encountered in divide
  np.divide(1.0, 0.0), np.divide(1.0, -0.0), np.divide(0.0, 0.0)


(inf, -inf, nan)

警告は初回だけ出力される。つまり、警告フィルターのエントリで `action` が `'once'` になっている。

Numpy でゼロ除算時の警告を制御するには、一般の警告と同様に `warnings.filterwarnings()` 関数を使用して警告フィルターの先頭にエントリを追加すればよいのであるが、 Numpy は便利な関数を用意している。

``` python
numpy.seterr(all=None, divide=None, over=None, under=None, invalid=None)
```

| 引数 | 意味 |
|:---|:---|
| `divide` | ゼロ除算の処理を以下の文字列で指定する。`None`（デフォルト）の場合、現在の動作は変更されない<br /><br />・`'ignore'`: 警告を出力しない<br /><br />・`'warn'`: `RuntimeWarning` を出力する<br /><br />・`'raise'`: `FloatingPointError` 例外を発生させる<br /><br />・`'call'`: `numpy.seterrcall()` 関数を使用して指定された関数を呼び出す<br /><br />・`'print'`: 警告を標準出力に直接出力する<br /><br />・`'log'`: `numpy.seterrcall()` で指定されたログオブジェクトにエラーを記録する |
| `over` | 浮動小数点のオーバーフロー時の処理を上記の文字列で指定する。`None`（デフォルト）の場合、現在の動作は変更されない |
| `under` | 浮動小数点のアンダーフロー時の処理を上記の文字列で指定する。`None`（デフォルト）の場合、現在の動作は変更されない |
| `invalid` | `0.0 / 0.0` の処理を上記の文字列で指定する。`None`（デフォルト）の場合、現在の動作は変更されない |
| `all` | ゼロ除算、浮動小数点のオーバーフローとアンダースロー、`0.0 / 0.0` の処理を上記の文字列でまとめて指定する。`None`（デフォルト）の場合、現在の動作は変更されない |

したがって、ゼロ除算時に `RuntimeWarning` を出したくなければ、あらかじめ

``` python
np.seterr(divide='ignore', invalid='ignore')
```

と書いておけばよいことになる。元に戻すには

``` python
np.seterr(divide='warn', invalid='warn')
```

とする。

警告を制御するためのコンテキストマネージャー `numpy.errstate` も用意されている。 `numpy.errstate` のコンストラクタは `all` 以外のキーワード引数を受け付ける。特定の除算だけ警告を出さないようにするには、以下のようにする。

``` python
with np.errstate(divide='ignore', invalid='ignore'):
    x = a / b
```

なお、 `numpy.seterrcall()` 関数の使い方については、[公式ドキュメント](https://numpy.org/doc/stable/reference/generated/numpy.seterrcall.html)を参照。

### 論理演算 ###

Numpy は、配列同士の論理演算をサポートしている。

以下の関数は、全てユニバーサル関数であり、同じ形状の配列同士で要素ごとに論理演算を評価した結果（`True` か `False`）を要素とする新しい配列（dtype は `numpy.bool_`）を返す。

| 演算 | 関数 | 演算子 | 機能 | 戻り値 |
|:---|:---|:--:|:---|:---|
| 論理積 | `logical_and` | `&` | 要素ごとに AND（論理積）を評価した結果 | 配列 |
| 論理和 | `logical_or` | &#124; | 要素ごとに OR（論理和）を評価した結果 | 配列 |
| 排他的論理和 | `logical_xor` | `^` | 要素ごとに XOR（排他的論理和）を評価した結果 | 配列 |
| 論理否定 | `logical_not` | `~` | 入力は 1 個で、要素ごとに NOT（論理否定）を評価した結果 | 配列 |

例えば、 2 つの配列 `a` と `b` に対して、 `numpy.logical_and(a, b)` と `a & b` は等価である。また、 3 つの配列 `a` と `b` と `c` に対して、 `numpy.logical_and.reduce([a, b, c])` は `numpy.logical_and(numpy.logical_and(a, b), c)` と同じで `a & b & c` と等価である。

`logical_not()` は入力が 1 個であり、 `logical_not(a)` と `~a` は等価である。

入力の各要素は真理値として評価される。

In [ ]:
a = np.array([True, True, False, False])
b = np.array([True, False, True, False])
a & b, a | b, a ^ b, ~a, a & b & ~a

(array([ True, False, False, False]),
 array([ True,  True,  True, False]),
 array([False,  True,  True, False]),
 array([False, False,  True,  True]),
 array([False, False, False, False]))

2 つの入力配列の形状が一致しない場合、ブロードキャスト可能なら、演算の前に暗黙的にブロードキャストが行われる。

In [ ]:
a = np.array([True, True, False, False])
a & False

array([False, False, False, False])

スカラー値同士で `&, |, ^, ~` は、 Python のビット演算となることに注意する。

### 比較演算 ###

Numpy は、配列同士の比較演算をサポートしている。

以下の関数は、同じ形状の配列同士で要素ごとに大小や等価性を比較し、その評価結果（`True` か `False`）を要素とする新しい配列（dtype は `numpy.bool_`）を返す。

| 演算 | 関数 | 演算子 | 機能 | 戻り値 |
|:---|:---|:--:|:---|:---|
| 大なり | `greater` | `>` | 要素ごとに `>` を評価した結果 | 配列 |
| 大なりイコール | `greater_equal` | `>=` | 要素ごとに `>=` を評価した結果 | 配列 |
| 小なり | `less` | `<` | 要素ごとに `<` を評価した結果 | 配列 |
| 小なりイコール | `less_equal` | `<=` | 要素ごとに `<=` を評価した結果 | 配列 |
| 等価性 | `equal` | `==` | 要素ごとに `==` を評価した結果 | 配列 |
| 非等価性 | `not_equal` | `!=` | 要素ごとに `!=` を評価した結果 | 配列 |

非数（NaN）との比較では常に `False` となる。

In [ ]:
np.greater([4, 2, 1], [2, 2, np.nan])

array([ True, False, False])

上記の関数は入力が 2 つの配列であるユニバーサル関数であり、暗黙的なブロードキャストをサポートする。対応する演算子も同様である。スカラー値と比較すると、その値とすべての要素がそれぞれ比較される。

In [ ]:
a = np.arange(9).reshape(3, 3)
a < 5

array([[ True,  True,  True],
       [ True,  True, False],
       [False, False, False]])

一方、**上記の関数はオブジェクトとして `reduce()` メソッドをサポートしていない**ことに注意する。 Python では `a < b < c` のように比較演算子を直接つなげることができるが、 Numpy 配列同士の比較演算子ではこのような書き方はできない。比較演算子で複数条件を指定したい場合は、それぞれの条件式を別々に書き `&` などの論理演算でつなげる方法を用いる必要がある。

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = np.array([7, 8, 9])
# np.greater.reduce([a, b, c]) や a < b < c のような書き方は不可
(a < b) & (b < c)

array([ True,  True,  True])

次のコードは、比較演算の結果をブーリアンインデックスに利用する例である。

In [ ]:
rng = np.random.default_rng()
a = rng.random((5, 5))
a, (a < .5)[0], a[:, (a < .5)[0]]

(array([[0.97597837, 0.14180546, 0.67503652, 0.91071182, 0.71749993],
        [0.72012836, 0.52573144, 0.20651588, 0.23902442, 0.43791389],
        [0.32118164, 0.46745595, 0.99283164, 0.40630831, 0.30150005],
        [0.73093661, 0.94985406, 0.27493701, 0.57216297, 0.77283029],
        [0.04096411, 0.83482705, 0.47359417, 0.53937537, 0.49683626]]),
 array([False,  True, False, False, False]),
 array([[0.14180546],
        [0.52573144],
        [0.46745595],
        [0.94985406],
        [0.83482705]]))

Numpy では、配列の間で `==` 演算子を使うと、`bool` 型の配列が返される。

In [ ]:
a = np.array([1, 2, 3])
a == a

array([ True,  True,  True])

配列同士が等しいときに単一の `True` を返す関数として、次の関数も提供されている。

``` python
numpy.array_equal(a1, a2, equal_nan=False)
```

非数（NaN）同士は等しいとはされないのであるが、この関数は、第 3 引数 `equal_nan` に `True` が指定されると、非数（NaN）同士を等しいものとみなす。

In [ ]:
a = np.array([1, 2, np.nan])
assert np.array_equal(a, a, equal_nan=True)

一方、 `==` では dtype が一致することは必要とされない。

In [ ]:
a = np.array([1, 2, 3], dtype=int)
b = np.array([1., 2., 3.], dtype=float)
a == b

array([ True,  True,  True])

配列の dtype が浮動小数点数の場合、 `numpy.equal()` 関数や `==` 演算子は扱いにくい。浮動小数点数の演算には丸め誤差が発生してしまうからである。2 つの浮動小数点数が許容誤差の範囲内なら「等しい」とするには、次の非ユニバーサル関数を使用する必要がある。

``` python
numpy.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)
```

配列 `a` の要素 `xa` と対応する配列 `b` の要素 `xb` の絶対差が、相対差 `(rtol * abs(xb))` と絶対差 `atol` を加算した値以下であるとき、`xa` と `xb` は許容誤差の範囲内とされる:

$$
abs(xa - xb) <= (atol + rtol * abs(xb))
$$

`equal_nan` 引数の意味は `numpy.array_equal()` の同名の引数と同じ。

In [ ]:
np.isclose([1e10, 1e-8], [1.00001e10, 1e-9])

array([ True,  True])

`numpy.isclose()` は、非ユニバーサル関数でありながら暗黙的なブロードキャストをサポートする。

In [ ]:
a = np.array([1e-8, 1e-7])
np.isclose(a, 0.0)  # np.isclose([1e-8, 1e-7], [0.0, 0.0]) と同じ

array([ True, False])

Numpy は、配列同士で全要素が許容誤差の範囲内のとき単一の `True` を返す関数も提供している。

``` python
numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)
```

この関数は、 `numpy.all(numpy.isclose(a, b, rtol, atol, equal_nan))` として実装されている。したがって、引数の意味は `numpy.isclose()` 関数と同じ。また、この実装のため、 `numpy.allclose` は、暗黙的なブロードキャストをサポートすることになって、  2 つの入力配列の次元数や形状が異なっていても `True` を返すことがあることに注意する。

In [ ]:
a = np.array([1e-8, 1e-9])
assert np.allclose(a, a)
assert np.allclose(a, 0.0)  # np.allclose([1e-8, 1e-7], [0.0, 0.0]) と同じ

Numpy は、単項の比較演算を行うユニバーサル関数も提供している。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `isnan` | 要素ごとに非数 `numpy.nan` であるかをテストし、結果を `bool` 型の配列として返す | 配列 |
| `isinf` | 要素ごとに無限大 `numpy.inf` であるかをテストし、結果を `bool` 型の配列として返す | 配列 |

In [ ]:
a = np.array([1., np.nan, np.inf])
np.isnan(a), np.isinf(a)

(array([False,  True, False]), array([False, False,  True]))

### 最大化・最小化 ###

Numpy は、 2 つの入力配列の両方に対して、要素ごとに大きな要素（または小さな要素）を持つ配列を作成する関数をサポートする。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `maximum` | 同じ形状の配列同士で要素ごとに比較して大きいほうを選んで新しい配列を返す。比較される要素の 1 つが非数（NaN）である場合、常に非数のほうを選ぶ | 配列 |
| `fmax` | `maximum` と似ているが、比較される要素の 1 つが非数（NaN）である場合は常に非数でないほうを選ぶ | 配列 |
| `minimum` | 同じ形状の配列同士で要素ごとに比較して小さいほうを選んで新しい配列を返す。比較される要素の 1 つが非数（NaN）である場合、常に非数のほうを選ぶ | 配列 |
| `fmin` | `minimum` と似ているが、比較される要素の 1 つが非数（NaN）である場合は常に非数でないほうを選ぶ | 配列 |

`numpy.maximum()` と `numpy.fmax()` の違い、および、 `numpy.minimum()` と `numpy.fmin()` の違いは、次のコードで確かめられる。

In [ ]:
a = np.array([2, 3, 4, np.nan, 0, np.nan])
b = np.array([1, 5, 2, 0, np.nan, np.nan])
np.maximum(a, b), np.fmax(a, b), np.minimum(a, b), np.fmin(a, b)

(array([ 2.,  5.,  4., nan, nan, nan]),
 array([ 2.,  5.,  4.,  0.,  0., nan]),
 array([ 1.,  3.,  2., nan, nan, nan]),
 array([ 1.,  3.,  2.,  0.,  0., nan]))

上記の関数は入力が 2 つの配列であるユニバーサル関数であり、暗黙的なブロードキャストをサポートする。入力の 1 つをスカラー値にすると、配列の要素の最小値をスカラー値以上にしたり、配列の要素の最大値をスカラー値以下にすることができる。

In [ ]:
a = np.array([-0.1, 0.2, -0.3, 1.4])
np.maximum(a, 0), np.minimum(a, 1)

(array([0. , 0.2, 0. , 1.4]), array([-0.1,  0.2, -0.3,  1. ]))

### 最大公約数・最小公倍数 ###

Numpy は、 2 つの入力配列の要素ごとに最大公約数や最小公倍数を計算するユニバーサル関数を提供している。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `gcd` | 同じ形状の配列同士で要素ごとに最大公約数を計算する | 配列 |
| `lcm` | 同じ形状の配列同士で要素ごとに最小公倍数を計算する | 配列 |

3 つ以上の配列を処理したい場合は、関数の `reduce()` メソッドが使える。

In [ ]:
a = np.array([0, 2, 3, 6])
b = np.array([3, 4, 5, 15])
c = np.array([6, 8, 9, 9])

np.gcd.reduce([a, b, c])  # np.gcd(np.gcd(a, b), c) と同じ

array([3, 2, 1, 3])

### 三角関数 ###

Numpy は、入力配列の要素ごとに三角関数や逆三角関数を計算する単項のユニバーサル関数を提供している。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `sin` | 各要素が角度のラジアンを示す入力配列に対し、要素ごとに正弦（サイン）を計算する | 配列 |
| `cos` | 各要素が角度のラジアンを示す入力配列に対し、要素ごとに余弦（コサイン）を計算する | 配列 |
| `tan` | 各要素が角度のラジアンを示す入力配列に対し、要素ごとに正接（タンジェント）を計算する | 配列 |
| `arcsin` | 各要素が単位円上の y 座標を示す入力配列に対し、要素ごとに逆正弦（アークサイン）を計算する | 配列 |
| `arccos` | 各要素が単位円上の x 座標を示す入力配列に対し、要素ごとに逆余弦（アークコサイン）を計算する | 配列 |
| `arctan` | 各要素が単位円上の x 座標・ y 座標で `y/x` を示す入力配列に対し、要素ごとに逆正接（アークタンジェント）を計算する。入力の要素が負の値の時は y 座標が負<br />であるとみなす | 配列 |
| `degrees` | 各要素が角度のラジアンを示す入力配列に対し、要素ごとに度に変換する | 配列 |
| `rad2deg` | `degrees` と同等 | 配列 |
| `radians` | 各要素が角度の度を示す入力配列に対し、要素ごとにラジアンに変換する | 配列 |
| `deg2rad` | `radians` と同等 | 配列 |

正接については、二項関数 `numpy.arctan2()` も提供されている。 `numpy.arctan()` は、負の入力を常に y 座標が負であるとみなすので、第一象限と第四象限の範囲での角度（`-pi/2` から `pi/2`）しか取得できない。これに対して、 `numpy.arctan2(y, x)` は、 x 座標・ y 座標を個別に指定することになり、`y/x` の逆正接を計算するので、第二象限と第三象限の範囲での角度（`pi/2` から `pi`、`-pi` から `-pi/2` ）も取得できる。

In [ ]:
x = np.array([-1, +1, +1, -1])
y = np.array([-1, -1, +1, +1])
np.degrees(np.arctan(y/x)), np.degrees(np.arctan2(y, x))

(array([ 45., -45.,  45., -45.]), array([-135.,  -45.,   45.,  135.]))

### 丸め ###

以下は、丸めを行う単項のユニバーサル関数である。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `rint` | 配列の要素ごとに小数点以下を「偶数丸め」で丸める | 配列 |
| `ceil` | 配列の要素ごとに天井を計算する | 配列 |
| `floor` | 配列の要素ごとに床を計算する | 配列 |
| `trunc` | 配列の要素ごとに小数点以下を切り捨てる | 配列 |

In [3]:
a = np.array([0, np.pi/2, np.pi])
np.rint(np.sin(a)), np.rint(np.cos(a))

(array([0., 1., 0.]), array([ 1.,  0., -1.]))

In [ ]:
a = np.array([-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.0])
np.ceil(a), np.floor(a), np.trunc(a)

(array([-1., -1., -0.,  1.,  2.,  2.,  2.]),
 array([-2., -2., -1.,  0.,  1.,  1.,  2.]),
 array([-1., -1., -0.,  0.,  1.,  1.,  2.]))

任意の桁で「偶数丸め」をするには、次の非ユニバーサル関数・メソッドを使用する必要がある。

``` python
numpy.round(a, decimals=0, out=None)
ndarray.round(decimals=0, out=None)
```

例えば、 `a.round(decimals)` は `numpy.round(a, decimals)` と同じ。

| 引数 | 意味 |
|:---|:---|
| `a` | 入力。array-like オブジェクトを受け付ける |
| `decimals` | 四捨五入する小数点以下の桁数を指定する。デフォルトは `0`。負の場合、小数点の左側の桁数を指定する |
| `out` | 結果を保存する配列 |

In [ ]:
np.round([np.e, np.pi], 2)

array([2.72, 3.14])

また、Numpy は、浮動小数点数を小数点以下 8 桁で四捨五入して表示する。これは表示上の処理であって、値そのものが丸められたわけではない。浮動小数点数を表示する桁数は変更可能で、 `numpy.set_printoptions()` 関数で指定することができる。

In [ ]:
a = np.array([1.123456789])
np.set_printoptions(precision=4)
print(a)
np.set_printoptions(precision=8)  # デフォルト
print(a)

[1.1235]
[1.12345679]


### 絶対値・二乗・平方根 ###

以下は、絶対値・二乗・平方根を計算する単項のユニバーサル関数である。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `absolute` | 配列の要素ごとに絶対値を計算する。複素数型の入力に対応する | 配列 |
| `abs` | `absolute` の別名 | 配列 |
| `fabs` | 配列の要素ごとに絶対値を計算する。入力が整数型なら出力の dtype が `float` となる。複素数型の入力には `TypeError` が発生する | 配列 |
| `square` | 配列の要素ごとに二乗を計算する | 配列 |
| `sqrt` | 配列の要素ごとに負でない平方根を計算する | 配列 |

なお、 `numpy.ndarray` には特殊メソッド `__abs__()` が定義されており、 Python の組み込み関数 `abs()` の引数に指定することができる。

In [4]:
assert np.absolute(3+4j) == abs(3+4j) == 5.0
a = np.array([-1, 1])
np.abs(a), abs(a), np.fabs(a)

(array([1, 1]), array([1, 1]), array([1., 1.]))

In [ ]:
x = np.sqrt([2, 3, 4])
x.round(2), np.square(x)

(array([1.41, 1.73, 2.  ]), array([2., 3., 4.]))

### 指数関数・対数関数 ###

以下は、指数関数・対数関数を計算する単項のユニバーサル関数である。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `exp` | 配列の要素 $x$ ごとに自然指数関数 $e^x$ を計算する（$e$ はネイピア数） | 配列 |
| `log` | 配列の要素 $x$ ごとに自然対数 $\log_{e}x$ を計算する（$e$ はネイピア数） | 配列 |
| `log10` | 配列の要素 $x$ ごとに常用対数 $\log_{10}x$ を計算する | 配列 |
| `log2` | 配列の要素 $x$ ごとに二進対数 $\log_{2}x$ を計算する | 配列 |

In [ ]:
np.exp([0, 1, 2]), np.log([1, np.e, np.e**2]), np.log10([1, 10, 100]), np.log2([1, 2, 4])

(array([1.        , 2.71828183, 7.3890561 ]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]))

### 区分関数 ###

区分関数（piecewise function）とは、定義域を分割して、区分ごとに適用する関数を指定する関数である。例えば、ステップ関数は区分関数の 1 つである。

Numpy は区分関数を計算する非ユニバーサル関数を提供している。

``` python
numpy.piecewise(x, condlist, funclist, *args, **kw)
```

| 引数 | 意味 |
|:---|:---|
| `x` | 計算の対象とする配列 |
| `condlist` | 条件のリスト。各要素は `x` と同じ形状のブール型配列でなければならない |
| `funclist` | 各条件に対応する関数のリスト。`condlist[i]` が `True` の場合は `funclist[i](x)` が出力値として使用される。関数は配列またはスカラー値を返す必要がある。関数の代わりに<br />スカラーとすることもでき、その場合は定数関数 `lambda x: スカラー` とみなされる。`condlist` の長さより 1 つ多い関数は、全ての条件が `False` の場合に使用される |
| `args`, `kw` | `funclist` の中の関数に追加して渡される位置引数とキーワード引数を指定する |

例えば、次の関数は区分関数の 1 つである。

$$
f(x) =
\begin{cases}
0 & (x < 0) \\
1 & (x > 1) \\
x & (otherwise)
\end{cases}
$$

次のコードは、この関数を `numpy.piecewise()` を使って計算する例である。

In [3]:
a = np.linspace(-2, 2, 9).reshape(3, 3)
a, np.piecewise(a, [a < 0, a > 1], [0, 1, lambda x: x])

(array([[-2. , -1.5, -1. ],
        [-0.5,  0. ,  0.5],
        [ 1. ,  1.5,  2. ]]),
 array([[0. , 0. , 0. ],
        [0. , 0. , 0.5],
        [1. , 1. , 1. ]]))

``` python
numpy.clip(a, a_min, a_max, out=None, **kwargs)
```

`numpy.clip(a, a_min, a_max)` は、 `numpy.piecewise(a, [a < a_min, a > a_max], [a_min, a_max, lambda a: a])` と同じ。

In [ ]:
a = np.linspace(-2, 2, 9).reshape(3, 3)
a, np.clip(a, 0, 1)

(array([[-2. , -1.5, -1. ],
        [-0.5,  0. ,  0.5],
        [ 1. ,  1.5,  2. ]]),
 array([[0. , 0. , 0. ],
        [0. , 0. , 0.5],
        [1. , 1. , 1. ]]))

### 集合演算 ###

``` python
numpy.isin(element, test_elements, assume_unique=False, invert=False, *, kind=None)
```

この関数は、2 つの配列を受け取り、第 1 引数の配列の要素ごとに、第 2 引数の配列の要素に含まれているなら `True`、そうでないなら `False` となる配列を返す非ユニバーサル関数である。

| 引数 | 意味 |
|:---|:---|
| `element` | 調べたい対象の配列または array-like オブジェクト |
| `test_elements` | この中に含まれているかを判定する配列または array-like オブジェクト |
| `assume_unique` | 両方の配列が一意な値だけからなると仮定して高速化したいときに `True` を指定する。デフォルトは `False` |
| `invert` | `True` の場合、計算結果を反転させる。つまり、含まれているなら `False`、そうでないなら `True` となる配列を返す。デフォルトは `False` |
| `kind` | キーワード専用。使用するアルゴリズムを指定する<br /><br />・`'sort'`: `test_elements` をソートしてから探索する方法（メモリ効率が良いが、少し遅め）<br /><br />・`'table'`: `test_elements` をハッシュテーブルに変換して探索する方法（高速なことが多いが、メモリ多めに使う）。`assume_unique=True` の効果はなくなる<br /><br />・`None`: デフォルト。メモリ使用量が多くなければ `'table'` を選択 |

In [ ]:
element = np.array([[0, 2],
                    [4, 6]])
test_elements = [1, 2, 4, 8]
np.isin(element, test_elements), np.isin(element, test_elements, invert=True)

(array([[False,  True],
        [ True, False]]),
 array([[ True, False],
        [False,  True]]))

実は、 `numpy.ndarray` は `__contains__` メソッドをサポートしているので Python の組み込み演算子 `in` に対応している。このため、単一の値が含まれているかどうかは `in` を使っても調べることができる。

In [11]:
a = np.array([[0, 2],
              [4, 6]])
4 in a, np.isin([4], a)

(True, array([ True]))

しかし、 `numpy.isin()` 関数を使うほうが速い。以下のコードは、 Python の集合に対する `in` 演算子と、配列に対する `in` 演算子と、配列に対する `numpy.isin()` 関数の実行速度を比較している。

In [9]:
import time

# テストデータを作成
np.random.seed(0)
size = 100_000  # 要素数
elements = np.random.randint(0, 200_000, size)
test_elements = np.random.randint(0, 200_000, size // 10)  # 少なめの検索対象

# データ型を準備
elements_set = set(elements)
elements_array = elements  # NumPy配列そのまま

# --- ① 集合 set に対する in ---
start = time.perf_counter()
results_set = [el in elements_set for el in test_elements]
time_set = time.perf_counter() - start

# --- ② 配列 ndarray に対する in ---
start = time.perf_counter()
results_array_in = [el in elements_array for el in test_elements]
time_array_in = time.perf_counter() - start

# --- ③ NumPy配列に対して numpy.isin() ---
start = time.perf_counter()
results_isin = np.isin(test_elements, elements_array)
time_isin = time.perf_counter() - start

# --- 結果を表示 ---
print(f"集合(set)に対するin: {time_set:.6f} 秒")
print(f"配列(ndarray)に対するin: {time_array_in:.6f} 秒")
print(f"numpy.isin(): {time_isin:.6f} 秒")

集合(set)に対するin: 0.004179 秒
配列(ndarray)に対するin: 1.273177 秒
numpy.isin(): 0.001258 秒


### 行列積・内積 ###

`numpy.matmul()` は、配列同士の行列積・内積をサポートするユニバーサル関数である。演算子 `@` もサポートされ、 `numpy.matmul(a, b)` と `a @ b` は等価である。

2 次元配列 `a` と `b` の間の行列積 `a @ b` では、その出力配列の `[i, j]` 位置の要素が以下のようにして計算される。

  1. 配列 `a` 内の `i` 番目の行 `a[i]` と配列 `b` 内の `j` 番目の列 `b[:, j]` とで、順番に要素の積を取り
  2. それらを全て足し合わせた和とする

つまり、`(a @ b)[i, j]` 要素は、次のように計算される。

$$
(a\,@\,b)[i, j] = a[i, 0] * b[0, j] + a[i, 1] * b[1, j] + \cdots + a[i, k] * b[k, j] \quad (k == a.shape[1] - 1 == b.shape[0] - 1)
$$

行列積 `a @ b` では、左側の配列の `a.shape[1]`（ 1 軸方向のサイズ）と右側の配列の `b.shape[0]`（ 0 軸方向のサイズ）が一致している必要がある。出力配列の形状は、 `(a.shape[0], b.shape[1])` と一致する。例えば、形状がそれぞれ `(n, k)` と `(k, m)` である 2 次元配列同士の行列積では、出力配列の形状が `(n, m)` となる。 `out` 引数に指定する配列は、この形状に一致させる必要がある。

In [ ]:
# 2 x 3 行列
a = np.array([[1, 2, 3],
              [4, 5, 6]])
# 3 x 2 行列
b = np.array([[1, 2],
              [3, 4],
              [5, 6]])
a @ b
# 2 x 2 行列
# [[1 * 1 + 2 * 3 + 3 * 5, 1 * 2 + 2 * 4 + 3 * 6],
#  [4 * 1 + 5 * 3 + 6 * 5, 4 * 2 + 5 * 4 + 6 * 6]]]

array([[22, 28],
       [49, 64]])

`numpy.matmul()` 関数と `@` 演算子も、暗黙的なブロードキャストをサポートするが、その働きはほかのユニバーサル関数とはかなり異なる。

  1. 両方の引数が 2 次元配列の場合、ブロードキャストは行われない。左側の配列の `shape[1]` と右側の配列の `shape[0]` が一致しない場合は、`ValueError` 例外が発生する。
  2. いずれかの引数が 3 次元以上の配列である場合、最後の 2 つのインデックスに存在する行列のバッチ（束）として扱われ、それに応じてブロードキャストされる。
  3. 第 1 引数 `a` が 1 次元配列の場合、先頭に次元（軸）を 1 つ追加した形である `a[np.newaxis]` という 2 次元配列に形状が変更される。行列積の計算後、先頭に追加された軸が削除される（つまり `[0]` でスライスされる）。
  4. 第 2 引数 `b` が 1 次元配列の場合、末尾に次元（軸）を 1 つ追加した形である `b[:, np.newaxis]` という 2 次元配列に形状が変更される。行列積の計算後、末尾に追加された軸が削除される（つまり、`[:, 0]` でスライスされる）。

このように、ブロードキャストが行われるのは規則 2 が適用される場合、つまり、いずれかの引数が 3 次元以上の配列である場合に限られる。

規則 3 と規則 4 により、 `numpy.matmul()` 関数と `@` 演算子は、ベクトルと行列との積をサポートする。とくに、規則 4 では第 2 引数のベクトルが線形代数で言うところの縦ベクトルに変換される。

In [ ]:
a = np.array([1, 2, 3])
b = np.array([[1, 2],
              [3, 4],
              [5, 6]])
a @ b  # (a[np.newaxis] @ b)[0] と同じ
(a[np.newaxis] @ b)[0]

array([22, 28])

In [ ]:
a = np.array([[1, 2, 3],
              [4, 5, 6]])
b = np.array([1, 3, 5])
a @ b  # (a @ b[:, np.newaxis])[:, 0] と同じ

array([22, 49])

サイズが同じ $N$ であるベクトル同士の内積は、次のような計算となる。

$$
\mathbf{x} \cdot \mathbf{y} = x_1y_1 + x_2y_2 + x_3y_3 + \cdots + x_Ny_N
\qquad (x_i \in \mathbf{x}, \; y_i \in \mathbf{y})
$$

これは、 1xN 行列と Nx1 行列の積と同じ計算である。

`numpy.matmul()` 関数と `@` 演算子では、規則 3 と規則 4 が同時に適用されることにより、ベクトル同士の内積もサポートされる。

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
a @ b  # (a[np.newaxis] @ b[:, np.newaxis])[0, 0] と同じ

32

Numpy は 1 次元配列同士の内積をサポートする非ユニバーサル関数 `numpy.dot()` と `numpy.inner()` も提供している。1 次元配列 `a` と `b` の間では、`numpy.dot(a, b)` と `numpy.inner(a, b)` は `a @ b` と等価である。`dot()` は  `numpy.ndarray` のメソッドとしても提供されており、 `a.dot(b)` は `numpy.dot(a, b)` と等価である。実は、 `numpy.dot()` 関数は 2 次元以上の配列同士の行列積にも対応している。しかしながら、2 次元以上の配列に対しては、`numpy.matmul()` 関数や `@` 演算子が推奨されている。

`numpy.dot()` と `numpy.inner()` 関数は入力としてスカラーを受け付けるが、 `numpy.matmul()` 関数と `@` 演算子は、入力としてスカラーを受け付けないことに注意する。スカラー倍の計算には `*` を使用することが推奨されている。

`numpy.matmul()` 関数と `@` 演算子では、入力の 3 次元配列は行列のバッチとみなされるため、複数の行列積をまとめて計算する形になる。すなわち、最初の次元（`axis=0`）がバッチのインデックスとなり、同じバッチインデックスの行列同士で行列積が計算される。例えば、 3 次元配列 `a` と `b` の間の行列積 `a @ b` では、次の計算と同じである。

``` python
np.array([ a[0] @ b[0], a[1] @ b[1], ..., a[a.shape[0]-1] @ b[b.shape[0]-1] ])
```

In [ ]:
a = np.arange(8).reshape(2, 2, 2)
b = a[::-1, ::-1, ::-1]
a, b, a @ b  # a @ b は np.array([ a[0] @ b[0], a[1] @ b[1] ]) と同じ

(array([[[0, 1],
         [2, 3]],
 
        [[4, 5],
         [6, 7]]]),
 array([[[7, 6],
         [5, 4]],
 
        [[3, 2],
         [1, 0]]]),
 array([[[ 5,  4],
         [29, 24]],
 
        [[17,  8],
         [25, 12]]]))

### 行列式・固有値・固有ベクトル・逆行列 ###

行列に対する線形代数の関数は、行列積など一部の例外を除いて `numpy.linalg` サブパッケージで提供されている。いずれも、非ユニバーサル関数である。

``` python
numpy.linalg.det(a)
```

この関数は、第 1 引数の行列式（determinant）の値（スカラー）を返す。入力は正方行列（行と列のサイズが同じ行列）である必要がある。

$n$ 次正方行列の行列式 $\det A=(a_{i,j})$ は、次の式で定義される:

$$
\det A := \sum_{\sigma\in S_n} (\mathrm{sgn}\,\sigma) a_{1, \sigma(1)} a_{2, \sigma(2)} \cdots a_{n, \sigma(n)}
$$

$S_n$ は「$1$ から $n$ までの整数を並べ替えるパターンの全て」という意味である。パターンの 1 つで決まる対応を置換と呼ぶ。 $\sigma\in S_n$ は置換に $\sigma$ と名付けることを意味する。例えば、 $n=2$ の場合、 $(1,2)$ の置換は $(1,2)\mapsto(1,2)$ と $(1,2)\mapsto(2,1)$ の 2 つであり、 1 つ目の置換を $\sigma$ と名付ければ $\sigma(1)=1$ と $\sigma(2)=2$ 、 2 つ目の置換を $\sigma$ と名付ければ $\sigma(1)=2$ と $\sigma(2)=1$ となる。 $\mathrm{sgn}\,\sigma$ は、置換の符号と呼ばれ、置換 $\sigma$ において整数を 2 個ずつ入れ替えるのに必要な回数が偶数回なら符号 $+$ 、奇数回なら符号 $-$ を表す。例えば、置換 $(1,2)\mapsto(1,2)$ は 0 回なので符号が $+$ 、置換 $(1,2)\mapsto(2,1)$ は 1 回なので符号が $-$ となる。

2 次正方行列 $A$ の行列式は、次のようになる:

$$
\det A = a_{1,1}a_{2,2} - a_{1,2}a_{2,1}
$$

$n=3$ の場合、置換は 6 個ある。その 1 つ $(1,2,3)\mapsto(2,3,1)$ という置換では $1$ と $3$ を入れ替えてから $3$ と $2$ を入れ替えたものなので符号は $+$ 、 $(1,2,3)\mapsto(3,2,1)$ という置換では $1$ と $3$ を入れ替えたものなので符号は $-$。このように符号に注意すれば、3 次正方行列 $A$ の行列式は、次のようになる:

$$
\det A = a_{1,1}a_{2,2}a_{3,3} + a_{1,2}a_{2,3}a_{3,1} + a_{1,3}a_{2,1}a_{3,2} - a_{1,3}a_{2,2}a_{3,1} - a_{1,1}a_{2,3}a_{3,2} - a_{1,2}a_{2,1}a_{3,3}
$$

4 次以上の正方行列の行列式も同様にして計算される。 $n$ 次に対して置換の個数は $n!=n\times(n-1)\times(n-2)\times\cdots\times2\times1$ となる。したがって、 $n$ 次正方行列の行列式は $n!$ 項の和となり、各項では $n$ 個の要素の積となる。このため、定義通りの行列式の計算量は $O(nn!)$ となる。実際の計算では効率化により計算量 $O(n^3)$ となるアルゴリズムが使われる。

Numpy の配列では添字の番号が 0 から始まることに注意する。例えば、上記の $a_{1,1}$ には配列の要素 `a[0, 0]` が対応する。

In [ ]:
a = np.array([[1., 2.],
              [3., 4.]])
assert np.allclose(np.linalg.det(a), -2.0)  # a の行列式は 1.0 * 4.0 - 2.0 * 3.0 = -2.0

``` python
numpy.linalg.eig(a)
```

この関数は、第 1 引数である `n` 次正方行列に対して、`n` 個の固有値（eigenvalue）が要素となる 1 次元配列と、各列が各固有値に対応する固有ベクトル（eigenvector）となる 2 次元配列からなる名前付きタプルを返す。固有値の 1 次元配列には、戻り値の `[0]` または名前 `eigenvalues` でアクセスできる。固有ベクトルの 2 次元配列には、戻り値の `[1]` または名前 `eigenvectors` でアクセスできる。入力は正方行列である必要がある。

線形代数において、固有値、固有ベクトルの定義は以下のようになる。$n$ 次正方行列 $A$ に対して、次の方程式

$$
A\mathbf{x} = \lambda\mathbf{x}
$$

を満たす零ベクトルでないベクトル $\mathbf{x}$ とスカラー $\lambda$ が存在するとき、 $\mathbf{x}$ を $A$ の固有ベクトル、$\lambda$ を $A$ の固有値と呼ぶ。

固有値と固有ベクトルを幾何学的に説明すると、以下のようになる。行列とベクトルの積がベクトルとなることから、行列とベクトルの積ではベクトルの変換が行われると見ることができるのであるが、行列が、ある向きのベクトルに対しては向きを変えずに長さだけ変える（あるいは長さも変えない）形に変換するときに、そのベクトルを固有ベクトル、長さを伸縮させる量（拡大率）を固有値と呼んでいる。

このように固有ベクトルは向きによって決まり、長さについては任意性がある。ふつう、固有ベクトルの長さは 1 に規格化される。

簡単な例を示す。 2 次正方行列 `array([[2., 0.], [0., 3.]])` は、 x 軸と平行なベクトル `array([1., 0.])` を `array([2., 0])` に変換するので `2` が固有値で `array([1., 0])` が固有ベクトルである。また、y 軸と平行なベクトル `array([0., 1.])` を `array([0., 3.])` に変換するので `3` が固有値で `array([0., 1.])` が固有ベクトルである。幾何学的には、 `array([[2., 0.], [0., 3.]])` は画像を x 軸方向に 2 倍、y 軸方向に 3 倍に拡大する変換行列である。

$n$ 次正方行列 $A$ に対して固有値を求める問題は、簡単な行列では直感的にわかるが、一般には次の $n$ 次方程式（固有方程式または特性方程式）を解く必要がある。

$$
\det(\lambda I - A) = 0 \qquad (I \,\text{は単位行列})
$$

上の例では、次の方程式を解いたことになる:

$$
\det\left(\begin{bmatrix}\lambda&0.\\0.&\lambda\end{bmatrix} - \begin{bmatrix}2.&0.\\0.&3.\end{bmatrix}\right)
= \det\begin{bmatrix}\lambda-2.&0.\\0.&\lambda-3.\end{bmatrix}
= (\lambda - 2.)(\lambda - 3.) = 0
$$

In [ ]:
a = np.array([[2., 0.],
              [0., 3.]])
assert np.allclose(a @ np.array([1., 0.]), np.array([2., 0.]))
assert np.allclose(a @ np.array([0., 1.]), np.array([0., 3.]))
eigenvalues, eigenvectors = np.linalg.eig(a)
eigenvalues, eigenvectors

(array([2., 3.]),
 array([[1., 0.],
        [0., 1.]]))

``` python
numpy.linalg.inv(a)
```

この関数は、第 1 引数の逆行列を返す。逆行列とは、もとの行列との行列積が単位行列になるような行列である。入力は正方行列である必要がある。逆行列の形状は、もとの行列と同じである。

幾何学的には、逆行列はもとの行列に対して逆向きの変換となる。例えば、正方行列 $A$ の逆行列を $A^{-1}$ と書くと、 $AA^{-1}=I$ であり、 $A\mathbf{x}=\mathbf{y}$ に対して $A^{-1}\mathbf{y}=\mathbf{x}$ が成り立つ。なぜなら、$A^{-1}\mathbf{y}=\mathbf{x}$ の両辺に左から $A$ を掛けると $AA^{-1}\mathbf{y}=A\mathbf{x}$ であり、左辺は $AA^{-1}\mathbf{y}=I\mathbf{y}=\mathbf{y}$ となるからである。

$n$ 次正方行列 $A$ の逆行列 $A^{-1}$ は、 $A$ の行列式 $\det A$ と[余因子行列](https://ja.wikipedia.org/wiki/余因子行列) $\widetilde A$ を使って次の計算により得られることが知られている。

$$
A^{-1} = \dfrac{1}{\det A} \widetilde A
$$

しかしながら、この計算方法では計算量が $O(n!)$ となることが知られている。 `numpy.linalg.inv()` 関数は LU 分解を利用するアルゴリズムを採用することで、計算量を $O(n^3)$ としている。

逆行列の計算式からわかることだが、行列式が 0 である正方行列（特異行列または退化行列）は逆行列を持たない。逆行列が存在しない行列に対しては、 `linalg.inv()` 関数はエラーとなる。

In [ ]:
a = np.array([[1., 2.], [3., 4.]])
ainv = np.linalg.inv(a)
assert np.allclose(np.dot(a, ainv), np.eye(2))
assert np.allclose(np.dot(ainv, a), np.eye(2))
a, ainv

(array([[1., 2.],
        [3., 4.]]),
 array([[-2. ,  1. ],
        [ 1.5, -0.5]]))

データ分析
----------

以下の関数は、第 1 引数に配列（または array-like オブジェクト）を受け付け、第 2 引数に配列の軸を指定する整数を受け付けて、指定された軸に沿って操作を行い、配列を出力する。ベクトル化されているが、全て非ユニバーサル関数である。 `numpy.ndarray` の同名のメソッドも提供されている。メソッドは、第 1 引数に軸を指定する整数を受け付ける。

引数として `out` や `where` を伴う関数もあり、それらの引数の意味は、ユニバーサル関数と共通する。

### ソート ###

``` python
numpy.sort(a, axis=-1, kind=None, order=None)
ndarray.sort(axis=-1, kind=None, order=None)
```

配列を `axis` で指定した軸に沿ってソートする。 `axis` のデフォルトは `-1` で、最後の軸に沿ってソートする。ただし、 `numpy.sort()` 関数と `ndarray.sort()` メソッドでは、次のような違いがある。

  * `numpy.sort(a, axis)` では配列 `a` 自体は変更されず、ソートされた配列が返される。これに対して、 `a.sort(axis)` では配列 `a` がインプレースでソートされ、`None` が返される。
  * `numpy.sort()` 関数の `axis` 引数が `None` の場合、入力を 1 次元配列に変換した上でソートする。これに対して、`ndarray.sort()` メソッドでは `axis` 引数は `None` を受け付けない。

Numpy のソートアルゴリズムは、デフォルトでは[クイックソート](https://ja.wikipedia.org/wiki/クイックソート)であり、[安定ソート](https://ja.wikipedia.org/wiki/安定ソート)ではない。`kind` 引数に `'stable'` または `'mergesort'` を指定すると、安定ソートである[マージソート](https://ja.wikipedia.org/wiki/マージソート)でソートが行われる。

In [ ]:
a = np.array([[9, 1, 4],
              [5, 8, 3],
              [2, 7, 6]])
b = np.sort(a)  # デフォルトでは最後の軸（列）に沿って各行をソートする
c = np.sort(a, 0)  # 0軸（行）に沿って各列をソートする
d = np.sort(a, None)  # axis に None を指定すると 1 次元配列に変換した上でソートする
b, c, d

(array([[1, 4, 9],
        [3, 5, 8],
        [2, 6, 7]]),
 array([[2, 1, 3],
        [5, 7, 4],
        [9, 8, 6]]),
 array([1, 2, 3, 4, 5, 6, 7, 8, 9]))

`numpy.sort()` 関数と `ndarray.sort()` メソッドには、 Python 標準の `sorted()` 関数や `sort()` メソッドのように引数 `reverse` は存在しない。降順にしたい場合は、インデックス参照で `axis` の軸（次元）に対してスライス `::-1` を使い、他の軸では `:` を使う（末尾の `,:` は省略可能なので、例えば `[::-1, :]` は `[::-1]` と同じ）。

In [ ]:
a = np.array([[9, 1, 4],
              [5, 8, 3],
              [2, 7, 6]])
b = np.sort(a)[:, ::-1]  # デフォルトでは最後の軸（列）に沿って各行をソートする
c = np.sort(a, 0)[::-1]  # 0軸（行）に沿って各列をソートする
d = np.sort(a, None)[::-1]  # axis に None を指定すると 1 次元配列に変換した上でソートする
b, c, d

(array([[9, 4, 1],
        [8, 5, 3],
        [7, 6, 2]]),
 array([[9, 8, 6],
        [5, 7, 4],
        [2, 1, 3]]),
 array([9, 8, 7, 6, 5, 4, 3, 2, 1]))

``` python
numpy.argsort(a, axis=-1, kind=None, order=None)
ndarray.argsort(axis=-1, kind=None, order=None)
```

配列を `axis` で指定した軸に沿ってソートするが、ソート後の配列は要素が値ではなく軸に沿ったもとのインデックスとなる。引数の意味は `numpy.sort()` と同じ。 `a.argsort(axis)` と `numpy.argsort(a, axis)` は全く同じであり、 `ndarray.argsort()` メソッドは配列自身を変更せず、配列を出力する。また、`axis` 引数が `None` を受け付ける。

In [ ]:
a = np.array([[9, 1, 4],
              [5, 8, 3],
              [2, 7, 6]])
b = np.argsort(a)  # デフォルトでは最後の軸（列）に沿って各行をソートする
c = np.argsort(a, 0)  # 0軸（行）に沿って各列をソートする
d = np.argsort(a, None)  # axis に None を指定すると 1 次元配列に変換した上でソートする
b, c, d

(array([[1, 2, 0],
        [2, 0, 1],
        [0, 2, 1]]),
 array([[2, 0, 1],
        [1, 2, 0],
        [0, 1, 2]]),
 array([1, 6, 5, 2, 3, 8, 7, 4, 0]))

`argsort` の結果をファンシーインデックスに利用すると、行列の特定の行や列をソートした順番を基準に他の行や列も並べ替えることができる。

In [ ]:
a = np.array([[9, 1, 4],
              [5, 8, 3],
              [2, 7, 6]])
b = a[:, np.argsort(a)[0]]  # 0 番目の行をソートした順番を基準に他の行も並べ替える
c = a[np.argsort(a, axis=0)[:, 2]]  # 2 番目の列をソートした順番を基準に他の列も並べ替える
b, c

(array([[1, 4, 9],
        [8, 3, 5],
        [7, 6, 2]]),
 array([[5, 8, 3],
        [9, 1, 4],
        [2, 7, 6]]))

【NumPy 2.0 での変更点】  
関数 `numpy.sort()` と `numpy.argsort()` には、キーワード専用引数 `stable` が追加され、 `stable=True` の指定は `kind='stable'` の指定と同等。 `stable` のデフォルトは `None`。

### 探索 ###

``` python
numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)
numpy.argmin(a, axis=None, out=None, *, keepdims=<no value>)
ndarray.argmax(axis=None, out=None, *, keepdims=False)
ndarray.argmin(axis=None, out=None, *, keepdims=False)
```

`axis` で指定した軸に沿った最大値（または最小値）のインデックスからなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。`axis` が `None`（デフォルト）の場合、入力配列を 1 次元配列に変換した上で最大値（または最小値）のインデックスを返す（出力の形式はスカラーとなる）。

キーワード専用引数 `keepdims` に `True` を指定した場合、削減された次元が追加されて出力配列が入力配列と同じ次元数を保持する。

In [ ]:
a = np.array([[9, 1, 4],
              [5, 8, 3],
              [2, 7, 6]])
b = np.argmax(a, axis=0)  # 0軸（行）に沿って（つまり各列で)最大値のインデックスからなる 1 次元配列を返す
c = np.argmax(a, axis=0, keepdims=True)  # np.argmax(a, axis=0)[np.newaxis] と同じ
d = np.argmax(a, axis=1)  # 1軸（列）に沿って（つまり各行で)最大値のインデックスからなる 1 次元配列を返す
e = np.argmax(a, axis=1, keepdims=True)  # np.argmax(a, axis=1)[:, np.newaxis] と同じ
b, c, d, e, np.argmax(a)

(array([0, 1, 2]),
 array([[0, 1, 2]]),
 array([0, 1, 1]),
 array([[0],
        [1],
        [1]]),
 0)

最大値（または最小値）が複数存在する場合は、最初に現れたデータのインデックスが選ばれる。

データに欠損値 `numpy.nan` が含まれていると、常に最初に現れた `numpy.nan` のインデックスが選ばれることに注意する。

In [ ]:
a = np.array([1., 1., np.nan, 3., np.nan, 5., 5.])
a.argmax(), a.argmin()

(2, 2)

欠損値 `numpy.nan` を無視して最大値（または最小値）のインデックスが選ばれるようにするには、次の関数を使う（`numpy.ndarray` のメソッドはない）。

``` python
numpy.nanargmax(a, axis=None, out=None, *, keepdims=<no value>)
numpy.nanargmin(a, axis=None, out=None, *, keepdims=<no value>)
```

In [ ]:
a = np.array([1., 1., np.nan, 3., np.nan, 5., 5.])
np.nanargmax(a), np.nanargmin(a)

(5, 0)

``` python
numpy.where(condition, [x, y, ]/)
```

Python の条件式 `x if condition else y` の配列版である。 `numpy.where(condition, x, y)` と呼び出すと、配列 `condition` の要素が真ならば配列 `x` の対応する要素が選ばれ、そうでなければ配列 `y` の対応する要素が選ばれた新しい配列が返される。第 1 引数 `condition` と第 2 引数 `x`、第 3 引数 `y` は array-like オブジェクトでもよい。

In [ ]:
np.where([[True, False],
          [True, True]],
         [[1, 2],
          [3, 4]],
         [[9, 8],
          [7, 6]])

array([[1, 8],
       [3, 4]])

第 1 引数 `condition` と第 2 引数 `x`、第 3 引数 `y` はそれぞれ配列の式でもよい。

In [ ]:
a = np.arange(9).reshape(3, 3)
b = np.where(a < 4, a * 10, a + 100)
a, b

(array([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]),
 array([[  0,  10,  20],
        [ 30, 104, 105],
        [106, 107, 108]]))

`numpy.where()` 関数は、非ユニバーサル関数でありながら暗黙的なブロードキャストをサポートする。第 1 引数 `condition` と第 2 引数 `x`、第 3 引数 `y` はブロードキャスト可能でなければならない。

In [ ]:
rng = np.random.default_rng()
a = rng.uniform(-3, 3, 6).reshape(2, 3)
b = np.where(a < 0.0, 0.0, a)
a, b

(array([[-1.99958818, -0.20633027, -2.5308997 ],
        [ 0.99251685,  2.14775001, -1.61287608]]),
 array([[0.        , 0.        , 0.        ],
        [0.99251685, 2.14775001, 0.        ]]))

第 2 引数 `x`、第 3 引数 `y` を省略すると、 `numpy.where()` 関数は入力 `condition` で真と評価される要素を抽出し、そのインデックスを表現するための配列のタプルを返す。 `condition` が 2 次元配列の場合、要素のインデックスは 2 つの軸で表現するから、 `numpy.where()` 関数は 2 つの配列からなるタプルを返す。

In [ ]:
a = np.arange(9).reshape(3, 3)
# [[0, 1, 2],
#  [3, 4, 5],
#  [6, 7, 8]]
np.where(a < 4)  # 真となる要素のインデックス [0, 0], [0, 1], [0, 2], [1, 0] を表す配列のタプルを返す

(array([0, 0, 0, 1]), array([0, 1, 2, 0]))

``` python
numpy.nonzero(a)
ndarray.nonzero()
```

`numpy.nonzero(a)` は、 `numpy.where(a != 0)` と同じ。

In [ ]:
a = np.arange(9).reshape(3, 3)
np.nonzero(a)

(array([0, 0, 1, 1, 1, 2, 2, 2]), array([1, 2, 0, 1, 2, 0, 1, 2]))

入力で真と評価される要素のインデックスが欲しいなら、次の関数を使うほうがよい。

``` python
numpy.argwhere(a)
```

In [ ]:
a = np.arange(9).reshape(3, 3)
np.argwhere(a < 4)

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0]])

### カウント ###

``` python
numpy.count_nonzero(a, axis=None, *, keepdims=False)
```

`axis` で指定した軸に沿って `0` でない要素の個数からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。 `axis` が `None`（デフォルト）の場合、入力全体で `0` でない要素の個数を返す（出力の形式はスカラーとなる）。

キーワード専用引数 `keepdims` に `True` を指定した場合、削減された次元が追加されて出力配列が入力配列と同じ次元数を保持する。

In [ ]:
a = np.array([[0, 1, 7, 0],
              [3, 0, 2, 19]])
b = np.count_nonzero(a, axis=0)
c = np.count_nonzero(a, axis=0, keepdims=True)  # np.count_nonzero(a, axis=0)[np.newaxis] と同じ
d = np.count_nonzero(a, axis=1)
e = np.count_nonzero(a, axis=1, keepdims=True)  # np.count_nonzero(a, axis=1)[:, np.newaxis] と同じ
b, c, d, e, np.count_nonzero(a)

(array([1, 1, 2, 1]),
 array([[1, 1, 2, 1]]),
 array([2, 3]),
 array([[2],
        [3]]),
 5)

Python の `False` と `0`、 `True` と `1` はそれぞれ等価と評価されるので、 `numpy.count_nonzero()` は配列内の `True` をカウントすることにも利用できる。

In [ ]:
a = np.arange(9).reshape(3, 3)
# [[0, 1, 2],
#  [3, 4, 5],
#  [6, 7, 8]]
np.count_nonzero(a < 4, axis=0)

array([2, 1, 1])

### 論理関数 ###

``` python
numpy.all(a, axis=None, out=None, keepdims=<no value>, *, where=<no value>)
ndarray.all(axis=None, out=None, keepdims=False, *, where=True)
```

`axis` で指定された軸に沿った全ての要素が真と評価されたなら `True`、そうでなければ `False` となる要素からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。 `axis` が `None`（デフォルト）の場合、入力の要素全体が真か否かの真理値を返す（出力の形式はスカラーとなる）。 `keepdims` 引数の意味は `numpy.argmax()` の同名の引数と同じ。

In [ ]:
a = np.arange(9).reshape(3, 3)
# [[0, 1, 2],
#  [3, 4, 5],
#  [6, 7, 8]]
np.all(a < 4, axis=1)  # 1軸（列）に沿って（つまり各行で)要素が全て 4 未満であるかどうかテスト

array([ True, False, False])

``` python
numpy.any(a, axis=None, out=None, keepdims=<no value>, *, where=<no value>)
ndarray.any(axis=None, out=None, keepdims=False, *, where=True)
```

`axis` で指定された軸に沿って要素が 1 つでも真と評価されたなら `True`、そうでなければ `False` となる要素からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。 `axis` が `None`（デフォルト）の場合、入力の要素が 1 つでも真と評価されるか否かの真理値を返す（出力の形式はスカラーとなる）。 `keepdims` 引数の意味は `numpy.argmax()` の同名の引数と同じ。

In [ ]:
a = np.arange(9).reshape(3, 3)
# [[0, 1, 2],
#  [3, 4, 5],
#  [6, 7, 8]]
np.any(a < 4, axis=1)  # 1軸（列）に沿って（つまり各行で) 4 未満の要素が 1 つでもあるかどうかテスト

array([ True,  True, False])

論理関数の戻り値をブーリアンインデックスに使う例として、欠損値を含む行や列を削除するコードを示す。

In [ ]:
a = np.array([[1., 2., 3.],
              [4., np.nan, 6.],
              [7., 8., 9.]])
(
    a[~np.isnan(a).any(axis=0)],
    a[:, ~np.isnan(a).any(axis=1)],
)

(array([[1., 2., 3.],
        [7., 8., 9.]]),
 array([[1., 3.],
        [4., 6.],
        [7., 9.]]))

### 集約関数 ###

``` python
numpy.sum(a, axis=None, dtype=None, out=None, keepdims=False, initial=0, where=True)
ndarray.sum(axis=None, dtype=None, out=None, keepdims=False, initial=0, where=True)
numpy.nansum(a, axis=None, dtype=None, out=None, keepdims=False, initial=0, where=True)
```

`axis` で指定した軸に沿った合計と `initial`（デフォルト値は `0`）の和からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。計算対象に非数 `numpy.nan` が含まれる場合、 `sum()` は計算結果を常に `numpy.nan` とするのに対して、 `nansum()` は `numpy.nan` を `0` として扱い計算を行う。

`axis` が `None`（デフォルト）の場合、配列の全ての要素の合計と `initial` の和を計算した値（スカラー）を返す。

In [ ]:
a = np.array([[1,      2],
              [np.nan, 5]])
np.sum(a, axis=0), np.nansum(a, axis=0), np.nansum(a)

(array([nan,  7.]), array([1., 7.]), 8.0)

``` python
umpy.prod(a, axis=None, dtype=None, out=None, keepdims=False, initial=1, where=True)
ndarray.prod(axis=None, dtype=None, out=None, keepdims=False, initial=1, where=True)
numpy.nanprod(a, axis=None, dtype=None, out=None, keepdims=False, initial=1, where=True)
```

`axis` で指定した軸に沿った積に `initial`（デフォルト値は `1`）を掛けた値からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。計算対象に非数 `numpy.nan` が含まれる場合、 `prod()` は計算結果を常に `numpy.nan` とするのに対して、 `nanprod()` は `numpy.nan` を `1` として扱い計算を行う。

`axis` が `None`（デフォルト）の場合、配列の全ての要素の積に `initial` を掛けた値（スカラー）を返す。

In [ ]:
a = np.array([[1,      2],
              [np.nan, 5]])
np.prod(a, axis=0), np.nanprod(a, axis=0), np.nanprod(a)

(array([nan, 10.]), array([ 1., 10.]), 10.0)

``` python
numpy.max(a, axis=None, out=None, keepdims=False, initial=<no value>, where=True)
ndarray.max(axis=None, out=None, keepdims=False, initial=<no value>, where=True)
numpy.min(a, axis=None, out=None, keepdims=False, initial=<no value>, where=True)
ndarray.min(axis=None, out=None, keepdims=False, initial=<no value>, where=True)
numpy.nanmax(a, axis=None, out=None, keepdims=False, initial=<no value>, where=True)
numpy.nanmin(a, axis=None, out=None, keepdims=False, initial=<no value>, where=True)
```

`axis` で指定した軸に沿った最大値（または最小値）からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。計算対象に非数 `numpy.nan` が含まれる場合、 `max()` と `min()` は計算結果を常に `numpy.nan` とするのに対して、 `nanmax()` と `nanmin()` は `numpy.nan` を無視する。

`axis` が `None`（デフォルト）の場合、配列の全ての要素の最大値（または最小値）となる値（スカラー）を返す。

`max()` と `nanmax()` では、 `initial` 引数に与えた値が出力要素の最小値とされる。 `min()` と `nanmin()` では `initial` 引数に与えた値が出力要素の最大値とされる。 `initial` 引数が与えられていない場合、配列（のスライス）が空であると `ValueError` 例外が発生する。

なお、`numpy.amax()` と `numpy.amin()` は、それぞれ `numpy.max()` と `numpy.min()` の別名である。

In [ ]:
a = np.array([[1,      2],
              [np.nan, 5]])
np.max(a, axis=0), np.nanmax(a, axis=0), np.nanmax(a), np.min(a, axis=0), np.nanmin(a, axis=0), np.nanmin(a)

(array([nan,  5.]),
 array([1., 5.]),
 5.0,
 array([nan,  2.]),
 array([1., 2.]),
 1.0)

``` python
numpy.mean(a, axis=None, dtype=None, out=None, keepdims=False, *, where=True)
ndarray.mean(axis=None, dtype=None, out=None, keepdims=False, *, where=True)
numpy.nanmean(a, axis=None, dtype=None, out=None, keepdims=False, *, where=True)
```

`axis` で指定した軸に沿った平均値からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。計算対象に非数 `numpy.nan` が含まれる場合、 `mean()` は計算結果を常に `numpy.nan` とするのに対して、 `nanmean()` は `numpy.nan` を無視する。

`axis` が `None`（デフォルト）の場合、配列の全ての要素の平均値となる値（スカラー）を返す

In [ ]:
a = np.array([[0, 1, 2],
              [3, 4, np.nan],
              [6, 7, 8]])
np.mean(a, axis=0), np.nanmean(a, axis=0), np.nanmean(a)

(array([ 3.,  4., nan]), array([3., 4., 5.]), 3.875)

``` python
numpy.var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=False, *, where=True)
ndarray.var(axis=None, dtype=None, out=None, ddof=0, keepdims=False, *, where=True)
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=False, *, where=True)
ndarray.std(axis=None, dtype=None, out=None, ddof=0, keepdims=False, *, where=True)
numpy.nanvar(a, axis=None, dtype=None, out=None, ddof=0, keepdims=False, *, where=True)
numpy.nanstd(a, axis=None, dtype=None, out=None, ddof=0, keepdims=False, *, where=True)
```

`axis` で指定した軸に沿った分散（または標準偏差）からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。計算対象に非数 `numpy.nan` が含まれる場合、 `var()` と `std()` は計算結果を常に `numpy.nan` とするのに対して、 `nanvar()` と `nanstd()` は `numpy.nan` を無視する。

`axis` が `None`（デフォルト）の場合、配列の全ての要素の分散（または標準偏差）となる値（スカラー）を返す。

`ddof` 引数は、分散、標準偏差を計算する際の分母すなわち自由度との差分として使われる。つまり、自由度は `N - ddof`（`N` はデータ数）。`ddof` のデフォルト値は `0` である。不偏分散と不偏標準偏差を計算したい場合は、 `ddof` 引数に `1` を指定することにより自由度 `N - 1` を分母として計算するようになる。

In [ ]:
a = np.array([[0, 1, 2],
              [3, 4, np.nan],
              [6, 7, 8]])
np.var(a, axis=0), np.nanvar(a, axis=0), np.nanvar(a)

(array([ 6.,  6., nan]), array([6., 6., 9.]), 7.359375)

``` python
numpy.median(a, axis=None, out=None, overwrite_input=False, keepdims=False)
numpy.nanmedian(a, axis=None, out=None, overwrite_input=False, keepdims=False)
```

`axis` で指定した軸に沿った中央値からなる 1 つの配列に圧縮されて次元が削減された形の配列を返す。計算対象に非数 `numpy.nan` が含まれる場合、 `median()` は計算結果を常に `numpy.nan` とするのに対して、 `nanmedian()` は `numpy.nan` を無視する。

`axis` が `None`（デフォルト）の場合、配列の全ての要素の平均値となる値（スカラー）を返す。

In [ ]:
a = np.array([[10, 7, 4], [6, 9, np.nan], [3, 2, 1]])
a, np.median(a, axis=0), np.nanmedian(a, axis=0), np.nanmedian(a)

(array([[10.,  7.,  4.],
        [ 6.,  9., nan],
        [ 3.,  2.,  1.]]),
 array([ 6.,  7., nan]),
 array([6. , 7. , 2.5]),
 5.0)

`overwrite_input` 引数が `False`（デフォルト）の場合、中央値の計算のために、入力配列 `a` のコピーを作成し、コピーの要素をソートする。 `overwrite_input` が `True` の場合、入力配列 `a` をインプレースでソートして中央値を計算するため、メモリが節約される（副作用として `a` が完全または部分的にソートされる）。

In [ ]:
a = np.array([[10, 7, 4], [6, 9, np.nan], [3, 2, 1]])
m = np.nanmedian(a, overwrite_input=True)
m, a

(5.0,
 array([[ 1.,  2.,  3.],
        [ 4.,  6.,  7.],
        [ 9., 10.,  1.]]))

ファイルの読み書き
------------------

``` python
numpy.loadtxt(fname, dtype=<class 'float'>, comments='#', delimiter=None, converters=None, skiprows=0, usecols=None, unpack=False, ndmin=0, encoding='bytes',
              max_rows=None, *, quotechar=None, like=None)
```

この関数は、テキストファイルを読み取り、2 次元配列を生成する。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `fname` | 読み取るファイル。path-like オブジェクト、または、パス名（文字列またはバイト列）のリストかジェネレータを指定できる。拡張子が `.gz`, `.bz2` のファイルは最初に解凍される |
| `dtype` | 生成する配列の dtype |
| `comments` | コメントの開始を示すために使用される文字または文字のリスト。`None` はコメントがないことを意味する。デフォルトは `#` |
| `delimiter` | 値を区切るために使用される文字。`None`（デフォルト）の場合は空白 |
| `converters` | 値を変換するための 1 引数関数。列ごとに関数を指定するには、列番号をキーとして関数をマップする辞書を使用する（例: `{0: func1, 1: func2}`） |
| `skiprows` | 最初にスキップする行数。行数には空行とコメント行も含まれる。デフォルトは `0` でスキップしない |
| `usecols` | 読み取る列番号（`0` が最初）のシーケンス。`None`（デフォルト）の場合は全ての列を読み取る |
| `unpack` | `True` の場合、転置された配列を生成する。この場合、`x, y, z = loadtxt(...)` のようにアンパックすると、各変数に列の値が入った 1 次元配列が格納される |
| `ndmin` | `numpy.array()` 関数の `ndmin` 引数と同じ意味だが、有効な値は `0`（デフォルト）、`1`、`2` のみとされる |
| `encoding` | 入力ファイルのデコードに使用されるエンコーディング。デフォルトは `'bytes'` |
| `max_rows` | `skiprows` 行の後に読み取る行数。行数には空行とコメント行は含まれない。`None`（デフォルト）の場合は最後の行まで読み取る |
| `quotechar` | 引用符として使用される文字。引用符付きフィールド内で `quotechar` が二重に使用されていればエスケープシーケンスとして扱われ、 `quotechar` を文字して扱う。`None`（デ<br />フォルト）の場合は引用符をサポートしない |

In [ ]:
np.loadtxt("sample_data/california_housing_test.csv", delimiter=",", usecols=(0, 1, 2, 3, 4), skiprows=1, max_rows=3)

array([[-122.05,   37.37,   27.  , 3885.  ,  661.  ],
       [-118.3 ,   34.26,   43.  , 1510.  ,  310.  ],
       [-117.81,   33.78,   27.  , 3589.  ,  507.  ]])

``` python
numpy.genfromtxt(fname, dtype=<class 'float'>, comments='#', delimiter=None, skip_header=0, skip_footer=0, converters=None, missing_values=None,
                 filling_values=None, usecols=None, names=None, excludelist=None, deletechars=" !#$%&'()*+, -./:;<=>?@[\\]^{|}~", replace_space='_',
                 autostrip=False, case_sensitive=True, defaultfmt='f%i', unpack=None, usemask=False, loose=True, invalid_raise=True, max_rows=None,
                 encoding='bytes', *, ndmin=0, like=None)
```

この関数も、テキストファイルを読み取り 2 次元配列を生成するが、 `numpy.loadtxt()` 関数の機能に加えて、欠損データ処理の機能が加えられている。

引数が拡張されており、以下の点に注意する。

  * `skiprows` ではなく `skip_header` を使う。
  * ファイルの末尾で読み飛ばす行数を指定 `skip_footer` で指定できる。
  * `missing_values` に指定した文字列を欠損データとして扱う。
  * データが欠損しているときに穴埋めする値を `filling_values` に指定できる。デフォルトでは `numpy.nan` が使われる。

``` python
numpy.savetxt(fname, X, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
```

この関数は、 1 次元配列または 2 次元配列をテキストファイルとして保存する。引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `fname` | 読み取るファイル。パス名かファイルオブジェクトを指定できる。拡張子が `.gz` の場合、GZIP 圧縮方式で自動的に圧縮される |
| `X` | ファイルに保存する 1 次元配列または 2 次元配列 |
| `fmt` | 各要素を書き出す際の書式を % 変換指定で与える。デフォルトは `'%.18e'`（小数点以下 18 桁の指数表記の浮動小数点数） |
| `delimiter` | 値を区切るために使用される文字。デフォルトは空白 |
| `newline` | 改行文字。デフォルトは `\n` |
| `header` | ヘッダー（ファイルの先頭に書き込まれる文字列） |
| `footer` | フッター（ファイルの最後に書き込まれる文字列） |
| `comments` | ヘッダーとフッターをコメントとしてマークするために、その文字列の前に追加される文字列。デフォルトは `'#'` |
| `encoding` | 出力ファイルのエンコードに使用されるエンコーディング。`None`（デフォルト）の場合、`latin1` |

In [ ]:
import io
import numpy as np

a = np.linspace(0, 1, 8).reshape((2, 4))
print(f"{a!r}")
print("-" * 100)
with io.StringIO() as f:
    header = "A,B,C,D"
    np.savetxt(f, a, delimiter=",", header=header, comments="")
    f.seek(0)
    for line in f:
        print(line.rstrip())

array([[0.        , 0.14285714, 0.28571429, 0.42857143],
       [0.57142857, 0.71428571, 0.85714286, 1.        ]])
----------------------------------------------------------------------------------------------------
A,B,C,D
0.000000000000000000e+00,1.428571428571428492e-01,2.857142857142856984e-01,4.285714285714285476e-01
5.714285714285713969e-01,7.142857142857141906e-01,8.571428571428570953e-01,1.000000000000000000e+00
